In [1]:
import tkinter as tk
from tkinter import ttk
import pandas as pd
import numpy as np
from PIL import Image
from tkinter import filedialog
import warnings
warnings.filterwarnings("ignore")
import regex
import openpyxl
from openpyxl import load_workbook, formatting, styles
import os
import pycountry
pd.set_option('display.max_columns', None)
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment
from openpyxl import workbook
from openpyxl.utils import get_column_letter
import customtkinter 
import re
from tkinter import messagebox
from sys import platform
import threading

In [2]:
def change_appearance_mode_event(new_appearance_mode: str):
    customtkinter.set_appearance_mode(new_appearance_mode)

def change_scaling_event(new_scaling: str):
    new_scaling_float = int(new_scaling.replace("%", "")) / 100
    customtkinter.set_widget_scaling(new_scaling_float)
    
# Function to choose internal template
def choose_internal_template():
    global file_path
    file_path = filedialog.askopenfilename(filetypes=(("Excel files", ["*.xlsx","*.xlsm"]), ("All files", "*.*")))
    if file_path:  # Check if a file has been selected
        messagebox.showinfo("Success", "Internal Template uploaded successfully")
    else:
        messagebox.showwarning("Warning", "Please upload the Internal Template file")
    

def choose_internal_template_1():
    global file_path_1
    file_path_1 = filedialog.askopenfilename(filetypes=(("Excel files", ["*.xlsx","*.xlsm"]), ("All files", "*.*")))
    if file_path_1:  # Check if a file has been selected
        messagebox.showinfo("Success", "Internal Template uploaded successfully")
    else:
        messagebox.showwarning("Warning", "Please upload the Internal Template file")

# Function to choose customer template
def choose_customer_template():
    global c_file_path
    c_file_path = filedialog.askopenfilename(filetypes=(("Excel files", "*.xlsx"), ("All files", "*.*")))
    if c_file_path:  # Check if a file has been selected
        messagebox.showinfo("Success", "Customer Template uploaded successfully")
    else:
        messagebox.showwarning("Warning", "Please upload the customer template file")

def choose_gateway_file():
    global g_file_path
    g_file_path = filedialog.askopenfilename(filetypes=(("Excel files", "*.xlsx"), ("All files", "*.*"))) 
    if g_file_path:  # Check if a file has been selected
        messagebox.showinfo("Success", "Gateway File uploaded successfully")
    else:
        messagebox.showwarning("Warning", "Please upload the Gateway file")
    

def process_customer_template(customer_data, num_rows):
    new_columns = []
    flattened_columns = []

    for i in range(num_rows):
        new_columns.append(customer_data.iloc[i, :].fillna('').astype(str))

    customer_data.columns = pd.MultiIndex.from_frame(pd.concat(new_columns, axis=1))

    main_heading = ""
    temp = {}
    for column in customer_data.columns:
        main_column = column[0]
        sub_columns = []

        if main_column != "":
            main_heading = main_column
            temp = {}

        for j in range(1, num_rows):
            if column[j] != "":
                temp[j] = column[j]
            elif j in temp:
                column = list(column)
                column[j] = temp[j]
                column = tuple(column)
            sub_columns.append(column[j])

        if main_heading == "":
            joined_column = "_".join(sub_columns)
        else:
            joined_column = f'{main_heading}_{("_".join(sub_columns))}'
        flattened_columns.append(joined_column)

    customer_data.columns = flattened_columns
    
    customer_data = customer_data.iloc[num_rows:]
    
    customer_data.reset_index(drop=True, inplace=True)
    return customer_data



def column_treatment(columns):
    
    last_non_unnamed = '' if 'unnamed' in columns[0].lower() else columns[0]  # Initializing last named column
    new_columns = []  # Initializing new list of modified column names
    new_column = ''
    for column in columns:
        if 'unnamed' in column.lower():
            new_column = last_non_unnamed  # Modifying column name to match last named column
        else:
            new_column = column
            last_non_unnamed = new_column  # Updating last named column
        new_columns.append(new_column)

    return new_columns

def template_cleaning(df):
    # Drop the first column to remove NA
    df = df.drop(columns=df.columns[0])
    df.columns = column_treatment(df.columns)
    df.iloc[0] = df.iloc[0].astype(str)
    df.columns = ['_'.join(i) for i in zip(df.columns.get_level_values(0).tolist(), df.iloc[0, :].replace(np.nan, '').tolist())]
    df = df.drop(index=0)
    df.reset_index(drop=True, inplace=True)

    return df


def template_cleaning_1(df):
    global remaining_columns
    dropped_columns = []
    
    df = df.drop(columns=df.columns[0])
    df_without_remaining = df.copy()
    for col in df.columns:
        if col in remaining_columns:
            df_without_remaining.drop(columns=[col], inplace=True)
            dropped_columns.append(col)
    #print(dropped_columns)
    

    # Apply column flattening to df_without_remaining
    df_without_remaining.columns = column_treatment(df_without_remaining.columns)
    df_without_remaining.iloc[0] = df_without_remaining.iloc[0].astype(str)
    df_without_remaining.columns = ['_'.join(i) for i in zip(df_without_remaining.columns.get_level_values(0).tolist(), df_without_remaining.iloc[0, :].replace(np.nan, '').tolist())]
    df_without_remaining = df_without_remaining.drop(index=0)
    df_without_remaining.reset_index(drop=True, inplace=True)
    
    
    # Concatenate df_without_remaining with the original DataFrame containing remaining_columns
    cleaned_df = pd.concat([df_without_remaining, df[dropped_columns]], axis=1)
    #print(cleaned_df.columns)

    return cleaned_df





In [3]:
def transfer_data():    
    try:
        num_rows = int(num_rows_entry.get())
    except ValueError:
        tk.messagebox.showerror("Error", "Please enter a valid number of rows.")
        return


    global internal_column_dropdowns
    global processed_data
    global internal_template
    global gcr_og_data
    global gtt
    global selected_matches
    global drop_down_states
    global l_vars
    global visible_columns
    #global tab1_internal_columns
    #global ct_columns
   
            
    customer_file_path = c_file_path
    num_rows = int(num_rows_entry.get())
    skip_rows = int(skip_rows_entry.get())
    sheet_name = sheet_entry.get()
    

    if skip_rows > 0:
        skip_rows = skip_rows - 1
        if sheet_name:
            customer_data = pd.read_excel(customer_file_path, skiprows=skip_rows, sheet_name=sheet_name)
        else:
            customer_data = pd.read_excel(customer_file_path, skiprows=skip_rows)
    else:
        if sheet_name:
            customer_data = pd.read_excel(customer_file_path, sheet_name=sheet_name, header=None)
        else:
            customer_data = pd.read_excel(customer_file_path, header=None)

    processed_data = process_customer_template(customer_data, num_rows)
    #ct_columns = list(processed_data.columns)
    

    if file_path:
        internal_template = pd.read_excel(file_path, skiprows=2,sheet_name='Air Internal Template')
        internal_template = template_cleaning(internal_template)
        print("Internal template loaded successfully!")
       
        internal_columns = list(internal_template.columns)
        print(internal_columns)
        #tab1_internal_columns = internal_columns.copy()
        gtt=internal_template
        internal_column_dropdowns = []
        drop_down_states={}
        #selected_matches = {}

#         def update_selected_matches(column, value):
#             selected_matches[column] = value

        def clear_option_menu(menu_var, default_value):
            menu_var.set(default_value)
            
        remove_b=[]
        o_vars = []
        options = customtkinter.StringVar(frame_2)
        l_vars=[]
        
        
        columns_to_hide = ['Lane Information_Country House','ORIGIN PICKUP/TRUCKING_PER KG', 'ORIGIN PICKUP/TRUCKING_Markup ($)',' OTHER MANDATORY ORIGIN CHARGES_DESCRIPTION',' OTHER MANDATORY ORIGIN CHARGES 1_DESCRIPTION', ' OTHER MANDATORY ORIGIN CHARGES 2_DESCRIPTION',' OTHER MANDATORY ORIGIN CHARGES 3_DESCRIPTION',
        ' OTHER MANDATORY ORIGIN CHARGES 4_DESCRIPTION',' OTHER MANDATORY ORIGIN CHARGES 5_DESCRIPTION',' OTHER MANDATORY ORIGIN CHARGES 6_DESCRIPTION',' OTHER MANDATORY ORIGIN CHARGES 7_DESCRIPTION',
        ' OTHER MANDATORY ORIGIN CHARGES 8_DESCRIPTION',' OTHER MANDATORY ORIGIN CHARGES 9_DESCRIPTION',' OTHER MANDATORY ORIGIN CHARGES 10_DESCRIPTION',' OTHER MANDATORY ORIGIN CHARGES 11_DESCRIPTION',
        ' OTHER MANDATORY ORIGIN CHARGES 12_DESCRIPTION',' OTHER MANDATORY ORIGIN CHARGES 13_DESCRIPTION',' OTHER MANDATORY ORIGIN CHARGES 14_DESCRIPTION', 'AIRPORT TO AIRPORT (COST)_ATA MINIMUM',
        'AIRPORT TO AIRPORT (COST)_-45.0', 'AIRPORT TO AIRPORT (COST)_45.0', 'AIRPORT TO AIRPORT (COST)_100.0', 'AIRPORT TO AIRPORT (COST)_300.0', 'AIRPORT TO AIRPORT (COST)_500.0', 'AIRPORT TO AIRPORT (COST)_1000.0',
        'ATA Markup (%)_-45.0', 'ATA Markup (%)_45.0', 'ATA Markup (%)_100.0', 'ATA Markup (%)_300.0', 'ATA Markup (%)_500.0', 'ATA Markup (%)_1000.0', 'ATA Markup (%)_nan', 'ATA Markup ($)_-45.0', 'ATA Markup ($)_45.0', 'ATA Markup ($)_100.0', 'ATA Markup ($)_300.0', 'ATA Markup ($)_500.0', 'ATA Markup ($)_1000.0', 'ATA Markup ($)_nan',
        'OTHER MANDATORY ATA CHARGES_DESCRIPTION', 'OTHER MANDATORY ATA CHARGES 1_DESCRIPTION','OTHER MANDATORY ATA CHARGES 2_DESCRIPTION', 'OTHER MANDATORY ATA CHARGES 3_DESCRIPTION', 'OTHER MANDATORY ATA CHARGES 4_DESCRIPTION','OTHER MANDATORY ATA CHARGES 5_DESCRIPTION',
        'DESTINATION DELIVERY/TRUCKING_PER KG','DESTINATION DELIVERY/TRUCKING_FSC (%) ','DESTINATION DELIVERY/TRUCKING_Markup ($)',
        ' OTHER MANDATORY DESTINATION CHARGES_DESCRIPTION',' OTHER MANDATORY DESTINATION CHARGES 1_DESCRIPTION',' OTHER MANDATORY DESTINATION CHARGES 2_DESCRIPTION',' OTHER MANDATORY DESTINATION CHARGES 3_DESCRIPTION',
        ' OTHER MANDATORY DESTINATION CHARGES 4_DESCRIPTION',' OTHER MANDATORY DESTINATION CHARGES 5_DESCRIPTION',' OTHER MANDATORY DESTINATION CHARGES 6_DESCRIPTION',' OTHER MANDATORY DESTINATION CHARGES 7_DESCRIPTION',
        ' OTHER MANDATORY DESTINATION CHARGES 8_DESCRIPTION',' OTHER MANDATORY DESTINATION CHARGES 9_DESCRIPTION',' OTHER MANDATORY DESTINATION CHARGES 10_DESCRIPTION',' OTHER MANDATORY DESTINATION CHARGES 11_DESCRIPTION',
        ' OTHER MANDATORY DESTINATION CHARGES 12_DESCRIPTION',' OTHER MANDATORY DESTINATION CHARGES 13_DESCRIPTION',' OTHER MANDATORY DESTINATION CHARGES 14_DESCRIPTION']
        
        visible_columns = [col for col in internal_columns if col not in columns_to_hide]
        
        for i, column in enumerate(visible_columns):
            if column not in columns_to_hide:
                #label_var = customtkinter.StringVar(frame_2)
                label = customtkinter.CTkLabel(frame_2, text=column)
                label.grid(row=i, column=0, sticky="w", padx=10, pady=3)
                l_vars.append(label)

            dropdown_var = customtkinter.StringVar(frame_2)
            dropdown_var.set("")  # Set initial value to empty string

            dropdown = customtkinter.CTkOptionMenu(frame_2,fg_color="#4D148C",button_color="#4D148C",width=50, variable=dropdown_var, values=list(processed_data.columns))
            dropdown.grid(row=i, column=2, sticky="w", padx=0, pady=10)

            #dropdown.bind('<<ComboboxSelected>>', lambda event, col=column, var=dropdown_var: update_selected_matches(col, var))

            internal_column_dropdowns.append(dropdown)
            
            drop_down_states[column] = dropdown_var
            o_vars.append(dropdown)
        
        for i, col in enumerate(visible_columns):
            b3 = customtkinter.CTkButton(frame_2,fg_color="#4D148C",text='Remove', command=lambda i=i: clear_option_menu(o_vars[i], ""))
            b3.grid(row=i, column=1)
        
    else:
        print("Please select an internal template.")
    pass
    



In [4]:

def transfer_selected_matches():
    global selected_matches
    global drop_down_states
    global s
    s={ v.get():k for k, v in drop_down_states.items()}

    selected_matches = {}
    for i, col in enumerate(internal_template.columns):
        match = internal_column_dropdowns[i].get()
        if match:
            selected_matches[col] = match
            
    #return selected_matches        
    print("Matches Transferred Successfully")
    
    return s
    
   

In [5]:
def transfer_data_2():
    if num_rows_entry_.get().strip() == '':
        tk.messagebox.showerror("Error", "Please enter a valid number of rows.")
        return
    
    try:
        num_rows = int(num_rows_entry_.get())
    except ValueError:
        tk.messagebox.showerror("Error", "Please enter a valid number of rows.")
        return

    global internal_column_dropdowns_1
    global processed_data
    global internal_template
    global tab1_internal_columns
    global tab2_internal_columns
    global ct_columns
    global transfer_selected_matches
    global last_column_index
    global s
    s = {}
    global remaining_columns
    remaining_columns = {}
    customer_file_path = c_file_path
    num_rows = int(num_rows_entry_.get())
    skip_rows = int(skip_rows_entry_.get())
    sheet_name = sheet_entry_.get()
    
    
    if skip_rows > 0:
        skip_rows = skip_rows - 1
        if sheet_name:
            customer_data = pd.read_excel(customer_file_path, skiprows=skip_rows, sheet_name=sheet_name)
        else:
            customer_data = pd.read_excel(customer_file_path, skiprows=skip_rows)
    else:
        if sheet_name:
            customer_data = pd.read_excel(customer_file_path, sheet_name=sheet_name, header=None)
        else:
            customer_data = pd.read_excel(customer_file_path, header=None)
            
    processed_data = process_customer_template(customer_data, num_rows)
    ct_columns = list(processed_data.columns)
    processed_data.iloc[:,:]=''
    num_rows, num_columns = customer_data.shape
    last_column_index = num_columns - 1
    
    if file_path_1:
        internal_template_1 = pd.read_excel(file_path_1, skiprows=2,sheet_name='Air Internal Template')
        internal_template_1 = template_cleaning_1(internal_template_1)
        print("Internal template loaded successfully!")
        internal_columns_1 = list(internal_template_1.columns)
        tab1_internal_columns = internal_columns_1.copy()
        #column_index = internal_template_1.shape[1] - 1
        #print(column_index)

    if file_path:
        internal_template = pd.read_excel(file_path, skiprows=None)
        internal_template = template_cleaning_1(internal_template)
        print("Internal template loaded successfully!")
        internal_columns = list(internal_template.columns)
        
        def rem_nan(col):
            if col.endswith("_nan"):
                return col.replace("_nan", "")
            else:
                return col
        
        #internal_columns=[rem_nan(col) for col in internal_template.columns]
        internal_columns = [rem_nan(col) if index > 275 else col for index, col in enumerate(internal_template.columns)]
        internal_template.columns = internal_columns
        #internal_columns = list(internal_template.columns)
        tab2_internal_columns = internal_columns.copy()
        columns_to_hide = ['Lane Information_Country House', 'ORIGIN PICKUP/TRUCKING_PER KG', 'ORIGIN PICKUP/TRUCKING_Markup ($)',' OTHER MANDATORY ORIGIN CHARGES_DESCRIPTION',' OTHER MANDATORY ORIGIN CHARGES 1_DESCRIPTION', ' OTHER MANDATORY ORIGIN CHARGES 2_DESCRIPTION',' OTHER MANDATORY ORIGIN CHARGES 3_DESCRIPTION',
        ' OTHER MANDATORY ORIGIN CHARGES 4_DESCRIPTION',' OTHER MANDATORY ORIGIN CHARGES 5_DESCRIPTION',' OTHER MANDATORY ORIGIN CHARGES 6_DESCRIPTION',' OTHER MANDATORY ORIGIN CHARGES 7_DESCRIPTION',
        ' OTHER MANDATORY ORIGIN CHARGES 8_DESCRIPTION',' OTHER MANDATORY ORIGIN CHARGES 9_DESCRIPTION',' OTHER MANDATORY ORIGIN CHARGES 10_DESCRIPTION',' OTHER MANDATORY ORIGIN CHARGES 11_DESCRIPTION',
        ' OTHER MANDATORY ORIGIN CHARGES 12_DESCRIPTION',' OTHER MANDATORY ORIGIN CHARGES 13_DESCRIPTION',' OTHER MANDATORY ORIGIN CHARGES 14_DESCRIPTION', 'AIRPORT TO AIRPORT (COST)_ATA MINIMUM',
        'AIRPORT TO AIRPORT (COST)_-45.0', 'AIRPORT TO AIRPORT (COST)_45.0', 'AIRPORT TO AIRPORT (COST)_100.0', 'AIRPORT TO AIRPORT (COST)_300.0', 'AIRPORT TO AIRPORT (COST)_500.0', 'AIRPORT TO AIRPORT (COST)_1000.0',
        'ATA Markup (%)_-45.0', 'ATA Markup (%)_45.0', 'ATA Markup (%)_100.0', 'ATA Markup (%)_300.0', 'ATA Markup (%)_500.0', 'ATA Markup (%)_1000.0', 'ATA Markup (%)_nan', 'ATA Markup ($)_-45.0', 'ATA Markup ($)_45.0', 'ATA Markup ($)_100.0', 'ATA Markup ($)_300.0', 'ATA Markup ($)_500.0', 'ATA Markup ($)_1000.0', 'ATA Markup ($)_nan',
        'OTHER MANDATORY ATA CHARGES_DESCRIPTION', 'OTHER MANDATORY ATA CHARGES 1_DESCRIPTION','OTHER MANDATORY ATA CHARGES 2_DESCRIPTION', 'OTHER MANDATORY ATA CHARGES 3_DESCRIPTION', 'OTHER MANDATORY ATA CHARGES 4_DESCRIPTION','OTHER MANDATORY ATA CHARGES 5_DESCRIPTION',
        'DESTINATION DELIVERY/TRUCKING_PER KG','DESTINATION DELIVERY/TRUCKING_FSC (%) ','DESTINATION DELIVERY/TRUCKING_Markup ($)',
        ' OTHER MANDATORY DESTINATION CHARGES_DESCRIPTION',' OTHER MANDATORY DESTINATION CHARGES 1_DESCRIPTION',' OTHER MANDATORY DESTINATION CHARGES 2_DESCRIPTION',' OTHER MANDATORY DESTINATION CHARGES 3_DESCRIPTION',
        ' OTHER MANDATORY DESTINATION CHARGES 4_DESCRIPTION',' OTHER MANDATORY DESTINATION CHARGES 5_DESCRIPTION',' OTHER MANDATORY DESTINATION CHARGES 6_DESCRIPTION',' OTHER MANDATORY DESTINATION CHARGES 7_DESCRIPTION',
        ' OTHER MANDATORY DESTINATION CHARGES 8_DESCRIPTION',' OTHER MANDATORY DESTINATION CHARGES 9_DESCRIPTION',' OTHER MANDATORY DESTINATION CHARGES 10_DESCRIPTION',' OTHER MANDATORY DESTINATION CHARGES 11_DESCRIPTION',
        ' OTHER MANDATORY DESTINATION CHARGES 12_DESCRIPTION',' OTHER MANDATORY DESTINATION CHARGES 13_DESCRIPTION',' OTHER MANDATORY DESTINATION CHARGES 14_DESCRIPTION']
        visible_columns = [col for col in internal_columns if col not in columns_to_hide]
        
        internal_column_dropdowns_1 = []
        
        def clear_option_menu(menu_var, default_value):
            menu_var.set(default_value)
            
        remove_b=[]
        o_vars = []
        options = customtkinter.StringVar(frame_2)
        
        for i, col in enumerate(processed_data.columns):
            options = customtkinter.StringVar(frame_2_)
            options.set("")
            label = customtkinter.CTkLabel(frame_2_, text=col,wraplength=500)
            label.grid(row=i, column=0, sticky="w", padx=10, pady=3)

            dropdown = customtkinter.CTkOptionMenu(frame_2_, fg_color="#4D148C", button_color="#4D148C", width=50, variable=options, values=list(visible_columns))
            dropdown.set("")
            dropdown.bind('<<ComboboxSelected>>')
            dropdown.grid(row=i, column=2, sticky="w", padx=0, pady=10)
            o_vars.append(options)

            if col in s:
                dropdown.set(s[col])
            else:
                dropdown.set("")

            internal_column_dropdowns_1.append(dropdown)

            
            
            # Pre-fill the dropdown with selected matches from tab 1
#             if column in selected_matches:
#                 dropdown.set(selected_matches[column])
#             else:
#                 dropdown.set("")
                
#             internal_column_dropdowns_1.append(dropdown)
                
        for i, col in enumerate(processed_data.columns):
            b3 = customtkinter.CTkButton(frame_2_,fg_color="#4D148C",text='Remove', command=lambda i=i: clear_option_menu(o_vars[i], ""))
            b3.grid(row=i, column=1)
    
    
    
    return last_column_index
    pass
    


In [6]:
import json

def load_data_from_file(file_path):
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
            return data
    except FileNotFoundError:
        return None
    
    
def load_selected_matches():
    global selected_matches
    file_path = filedialog.askopenfilename(filetypes=[("JSON Files", "*.json")])
    if file_path:
        with open(file_path, 'r') as json_file:
            selected_matches = json.load(json_file)

        if selected_matches:
            # Reverse the dictionary to set dropdown values in reverse order
            reversed_matches = {v: k for k, v in selected_matches.items()}
            for i, col in enumerate(processed_data.columns):
                if col in reversed_matches:
                    internal_column_dropdowns_1[i].set(reversed_matches[col])
                    
    messagebox.showinfo("Success", "Matches uploaded successfully.")                

o_vars_1 = []
uploaded_matches = []
def upload_matches():
    global uploaded_matches
    global selected_matches
    global data
    global file_path
    global internal_column_dropdowns
    global processed_data
    global internal_template
    global visible_columns
    
    
    
    customer_file_path = c_file_path
    num_rows = int(num_rows_entry.get())
    skip_rows = int(skip_rows_entry.get())
    sheet_name = sheet_entry.get()
    
    

    if skip_rows > 0:
        skip_rows = skip_rows - 1
        if sheet_name:
            customer_data = pd.read_excel(customer_file_path, skiprows=skip_rows, sheet_name=sheet_name)
        else:
            customer_data = pd.read_excel(customer_file_path, skiprows=skip_rows)
    else:
        if sheet_name:
            customer_data = pd.read_excel(customer_file_path, sheet_name=sheet_name, header=None)
        else:
            customer_data = pd.read_excel(customer_file_path, header=None)

    processed_data = process_customer_template(customer_data, num_rows)
    
    if file_path:
        internal_template = pd.read_excel(file_path, skiprows=2,sheet_name='Air Internal Template')
        internal_template = template_cleaning(internal_template)
        print("Internal template loaded successfully!")
       
        internal_columns = list(internal_template.columns)
        #tab1_internal_columns = internal_columns.copy()

        internal_column_dropdowns = []
        drop_down_states={}
        
    def clear_option_menu(menu_var, default_value):
        menu_var.set(default_value)
        
    columns_to_hide = ['Lane Information_Country House', 'ORIGIN PICKUP/TRUCKING_PER KG', 'ORIGIN PICKUP/TRUCKING_Markup ($)',' OTHER MANDATORY ORIGIN CHARGES_DESCRIPTION',' OTHER MANDATORY ORIGIN CHARGES 1_DESCRIPTION', ' OTHER MANDATORY ORIGIN CHARGES 2_DESCRIPTION',' OTHER MANDATORY ORIGIN CHARGES 3_DESCRIPTION',
        ' OTHER MANDATORY ORIGIN CHARGES 4_DESCRIPTION',' OTHER MANDATORY ORIGIN CHARGES 5_DESCRIPTION',' OTHER MANDATORY ORIGIN CHARGES 6_DESCRIPTION',' OTHER MANDATORY ORIGIN CHARGES 7_DESCRIPTION',
        ' OTHER MANDATORY ORIGIN CHARGES 8_DESCRIPTION',' OTHER MANDATORY ORIGIN CHARGES 9_DESCRIPTION',' OTHER MANDATORY ORIGIN CHARGES 10_DESCRIPTION',' OTHER MANDATORY ORIGIN CHARGES 11_DESCRIPTION',
        ' OTHER MANDATORY ORIGIN CHARGES 12_DESCRIPTION',' OTHER MANDATORY ORIGIN CHARGES 13_DESCRIPTION',' OTHER MANDATORY ORIGIN CHARGES 14_DESCRIPTION', 'AIRPORT TO AIRPORT (COST)_ATA MINIMUM',
        'AIRPORT TO AIRPORT (COST)_-45.0', 'AIRPORT TO AIRPORT (COST)_45.0', 'AIRPORT TO AIRPORT (COST)_100.0', 'AIRPORT TO AIRPORT (COST)_300.0', 'AIRPORT TO AIRPORT (COST)_500.0', 'AIRPORT TO AIRPORT (COST)_1000.0',
        'ATA Markup (%)_-45.0', 'ATA Markup (%)_45.0', 'ATA Markup (%)_100.0', 'ATA Markup (%)_300.0', 'ATA Markup (%)_500.0', 'ATA Markup (%)_1000.0', 'ATA Markup (%)_nan', 'ATA Markup ($)_-45.0', 'ATA Markup ($)_45.0', 'ATA Markup ($)_100.0', 'ATA Markup ($)_300.0', 'ATA Markup ($)_500.0', 'ATA Markup ($)_1000.0', 'ATA Markup ($)_nan',
        'OTHER MANDATORY ATA CHARGES_DESCRIPTION', 'OTHER MANDATORY ATA CHARGES 1_DESCRIPTION','OTHER MANDATORY ATA CHARGES 2_DESCRIPTION', 'OTHER MANDATORY ATA CHARGES 3_DESCRIPTION', 'OTHER MANDATORY ATA CHARGES 4_DESCRIPTION','OTHER MANDATORY ATA CHARGES 5_DESCRIPTION',
        'DESTINATION DELIVERY/TRUCKING_PER KG','DESTINATION DELIVERY/TRUCKING_FSC (%) ','DESTINATION DELIVERY/TRUCKING_Markup ($)',
        ' OTHER MANDATORY DESTINATION CHARGES_DESCRIPTION',' OTHER MANDATORY DESTINATION CHARGES 1_DESCRIPTION',' OTHER MANDATORY DESTINATION CHARGES 2_DESCRIPTION',' OTHER MANDATORY DESTINATION CHARGES 3_DESCRIPTION',
        ' OTHER MANDATORY DESTINATION CHARGES 4_DESCRIPTION',' OTHER MANDATORY DESTINATION CHARGES 5_DESCRIPTION',' OTHER MANDATORY DESTINATION CHARGES 6_DESCRIPTION',' OTHER MANDATORY DESTINATION CHARGES 7_DESCRIPTION',
        ' OTHER MANDATORY DESTINATION CHARGES 8_DESCRIPTION',' OTHER MANDATORY DESTINATION CHARGES 9_DESCRIPTION',' OTHER MANDATORY DESTINATION CHARGES 10_DESCRIPTION',' OTHER MANDATORY DESTINATION CHARGES 11_DESCRIPTION',
        ' OTHER MANDATORY DESTINATION CHARGES 12_DESCRIPTION',' OTHER MANDATORY DESTINATION CHARGES 13_DESCRIPTION',' OTHER MANDATORY DESTINATION CHARGES 14_DESCRIPTION']
    visible_columns = [col for col in internal_columns if col not in columns_to_hide]
        
        
        
    selected_matches = {}
    file_path = filedialog.askopenfilename(filetypes=[("JSON Files", "*.json")])
    if file_path:
        with open(file_path, 'r') as file:
            data = json.load(file)  
        
        
    for b,(i, k) in enumerate(data.items()):
        if i in visible_columns:
                        label = customtkinter.CTkLabel(frame_2, text=i)
                        label.grid(row=b, column=0, sticky="w", padx=10, pady=3)

                        dropdown_var = customtkinter.StringVar(frame_2)
                        dropdown_var.set(k)  # Set value from uploaded_matches if available
                        o_vars_1.append(dropdown_var) # Append the StringVar

                        dropdown = customtkinter.CTkOptionMenu(frame_2, width=50, variable=dropdown_var,values=list(processed_data.columns))
                        dropdown.grid(row=b, column=2, sticky="w", padx=0, pady=10)
                        uploaded_matches.append(k)
                        internal_column_dropdowns.append(dropdown)
                        drop_down_states[i] = dropdown_var
                
    for i, (u,col) in enumerate(data.items()):
        if u in visible_columns:
                b3 = customtkinter.CTkButton(frame_2,fg_color="#4D148C",text='Remove', command=lambda i=i: clear_option_menu(o_vars_1[i], ""))
                b3.grid(row=i, column=1)        

               # Update the selected_matches dictionary with values from 'k'
                #selected_matches[k] = i  
        
    messagebox.showinfo("Success", "Matches uploaded successfully.")   
        #return selected_matches   
        
        

def transfer_all_data_3():
    global remaining_columns# Declare the global variable
    global visible_columns
    transferred_columns = [] 
    remaining_columns = None
    global internal_templat
    internal_templat=internal_template.iloc[0:0]
    for i, col in enumerate(o_vars_1):
        internal_column = visible_columns[i]
        customer_column= col.get()
        if internal_column and customer_column:
            internal_templat[internal_column] = processed_data[customer_column]   
            transferred_columns.append(customer_column)
    transferred_columns_set = set(transferred_columns)
    remaining_columns = processed_data[[col for col in processed_data.columns if col not in transferred_columns_set]]
    
    print("Transfer successful")
    messagebox.showinfo("Success", "Transfer Successful") 
    return remaining_columns
    pass

            
def save_selected_matches():
    file_path = filedialog.asksaveasfilename(defaultextension='.json', filetypes=[("JSON Files", "*.json")])
    if file_path:
        data =  {k: v.get() for k, v in drop_down_states.items()}  # Get values from the tkinter IntVar objects
        
        with open(file_path, 'w') as file:
            json.dump(data, file)
        print("Selected matches saved successfully.")
        messagebox.showinfo("Success","Matches saved successfully")


# Global variable to store column_names data
column_names = []

# Function to load data from a JSON file
def load_data_from_file(file_path):
    global selected_matches, drop_down_states
    if file_path:
        with open(file_path, 'r') as file:
            loaded_data = json.load(file)
        selected_matches = loaded_data.get('selected_matches', {})
        drop_down_states = loaded_data.get('drop_down_states', {})
        # Update dropdowns based on loaded data
        for col, state in loaded_data.items():
            drop_down_states[col] = tk.StringVar(value=state)
            # Update the dropdowns with the loaded data
            if col in column_names:
                drop_downs[col].config(textvariable=drop_down_states[col])
    else:
        messagebox.showerror("Error", "Please choose a valid file.")


In [7]:
def country_house_Column(x):
    if x['origin port'] != "" and x['destination port'] != "":
        return str(x['origin port']) + " " + str(x['destination port'])
    else:
        return ""
    pass 
    
    """This function takes a pandas DataFrame as input and returns a concatenated string of origin port and 
    destination port if both of them are not null, otherwise returns NaN.
    
    Args:
    x (DataFrame): A pandas DataFrame which should have columns named 'origin port' and 'destination port'
    
    Returns:
    str or NaN: A concatenated string of origin port and destination port or NaN
    """

    
#       if x['origin port'] != "" and x['destination port'] != "":
#         return str(x['origin port']) + " " + str(x['destination port'])
#       else:
#         return ""
#       pass  
#     if ((x['origin port'] != "") and (x['destination port'] != "")):
#         return str(x['origin port']) + " " + str(x['destination port'])
    
#     else:
#         return np.nan
#     pass
#     if pd.notna(x['origin port']) & pd.notna(x['destination port']):
#         return str(x['origin port']) + " " + str(x['destination port'])
#     else:
#         return np.nan
#     pass


#       if x['origin port'] != "" and x['destination port'] != "":
#         return str(x['origin port']) + " " + str(x['destination port'])
#       else:
#             return ""

In [8]:
def gcr(df,df1,df2):

    df['Origin  _Origin Gateway'] = df['Origin  _Origin Gateway'].astype(str)
    df1.columns = column_treatment(df1.columns)  
    df1.iloc[0] = df1.iloc[0].astype(str)
    df1.columns = ['_'.join(i) for i in zip(df1.columns.get_level_values(0).tolist(), df1.iloc[0,:].replace(np.nan,'').tolist())]
    df1 = df1.drop(index = 0)
    df1.drop_duplicates(subset=['Information_Origin Gateway Code'],keep='first',inplace=True)
    df1['Information_Origin Gateway Code'] = df1['Information_Origin Gateway Code'].astype(str)
    
    #Destination 
    df2.columns = column_treatment(df2.columns)  
    df2.iloc[0] = df2.iloc[0].astype(str)
    df2.columns = ['_'.join(i) for i in zip(df2.columns.get_level_values(0).tolist(), df2.iloc[0,:].replace(np.nan,'').tolist())]
    df2 = df2.drop(index = 0)
    df2.drop_duplicates(subset=['Information_Destination Gateway Code'],keep='first',inplace=True)
    df2['Information_Destination Gateway Code'] = df2['Information_Destination Gateway Code'].astype(str)
 
    df3 = pd.merge(df, df1, left_on = 'Origin  _Origin Gateway', right_on = 'Information_Origin Gateway Code', how = 'left')
    df3 = pd.merge(df3, df2, left_on = 'Destination  _Destination Gateway', right_on = 'Information_Destination Gateway Code', how = 'left')
    df3.to_clipboard()
    df3 = df3.loc[:, :'Dest Remarks_nan']    
    return df3
    



In [9]:
def transfer_all_data_2():
    global processed_dat
    processed_dat = processed_data.iloc[0:0]
    for i, col_dropdown in enumerate(internal_column_dropdowns_1):
        customer_column = processed_dat.columns[i]
        internal_column = col_dropdown.get()
        if internal_column and customer_column:
            processed_dat[customer_column]=internal_template[internal_column]
            
    print("Transfer successful")
    messagebox.showinfo("Success","Transfer Successful")
    pass

In [10]:
def additional_columns():
    global tab1_internal_columns, tab2_internal_columns, ct_columns
    additional_columns = list(set(tab2_internal_columns) - set(tab1_internal_columns)-set(ct_columns))
    
    # Get the index from the processed_data DataFrame
    index = processed_dat.index
    
    # Transfer additional columns from internal_template to processed_data
    for column in additional_columns:
        processed_dat[column] = internal_template[column]
    
    # Set the index back to the processed_data DataFrame
    processed_dat.index = index
    
    print("Additional Columns Transfer successful")
    messagebox.showinfo("Success","Additional Columns Transfer Successful")

In [11]:
def transfer_all_data():
    global remaining_columns
    global visible_columns
    transferred_columns = []
    remaining_columns = None
    global internal_templat
    internal_templat=internal_template.iloc[0:0]
    for i, col_dropdown in enumerate(internal_column_dropdowns):
        internal_column = visible_columns[i]
        customer_column = col_dropdown.get()    
        if internal_column and customer_column:
            internal_templat[internal_column] = processed_data[customer_column]   
            transferred_columns.append(customer_column)
    transferred_columns_set = set(transferred_columns)
    remaining_columns = processed_data[[col for col in processed_data.columns if col not in transferred_columns_set]]
    
    print("Transfer successful")
    messagebox.showinfo("Success","Transfer Successful")
    return remaining_columns
    pass

def truck_df_treatment():
    states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}
    
    
    country_codes = {
         'AFGHANISTAN': 'AF',
         'ÅLAND ISLANDS': 'AX',
         'ALBANIA': 'AL',
         'ALGERIA': 'DZ',
         'AMERICAN SAMOA': 'AS',
         'ANDORRA': 'AD',
         'ANGOLA': 'AO',
         'ANGUILLA': 'AI',
         'ANTARCTICA': 'AQ',
         'ANTIGUA AND BARBUDA': 'AG',
         'ARGENTINA': 'AR',
         'ARMENIA': 'AM',
         'ARUBA': 'AW',
         'AUSTRALIA': 'AU',
         'AUSTRIA': 'AT',
         'AZERBAIJAN': 'AZ',
         'BAHAMAS': 'BS',
         'BAHRAIN': 'BH',
         'BANGLADESH': 'BD',
         'BARBADOS': 'BB',
         'BELARUS': 'BY',
         'BELGIUM': 'BE',
         'BELIZE': 'BZ',
         'BENIN': 'BJ',
         'BERMUDA': 'BM',
         'BHUTAN': 'BT',
         'BOLIVIA, PLURINATIONAL STATE OF': 'BO',
         'BONAIRE, SINT EUSTATIUS AND SABA': 'BQ',
         'BOSNIA AND HERZEGOVINA': 'BA',
         'BOTSWANA': 'BW',
         'BOUVET ISLAND': 'BV',
         'BRAZIL': 'BR',
         'BRITISH INDIAN OCEAN TERRITORY': 'IO',
         'BRUNEI DARUSSALAM': 'BN',
         'BULGARIA': 'BG',
         'BURKINA FASO': 'BF',
         'BURUNDI': 'BI',
         'CAMBODIA': 'KH',
         'CAMEROON': 'CM',
         'CANADA': 'CA',
         'CAPE VERDE': 'CV',
         'CAYMAN ISLANDS': 'KY',
         'CENTRAL AFRICAN REPUBLIC': 'CF',
         'CHAD': 'TD',
         'CHILE': 'CL',
         'CHINA': 'CN',
         'CHRISTMAS ISLAND': 'CX',
         'COCOS (KEELING) ISLANDS': 'CC',
         'COLOMBIA': 'CO',
         'COMOROS': 'KM',
         'CONGO': 'CG',
         'CONGO, THE DEMOCRATIC REPUBLIC OF THE': 'CD',
         'COOK ISLANDS': 'CK',
         'COSTA RICA': 'CR',
         "CÔTE D'IVOIRE": 'CI',
         'CROATIA': 'HR',
         'CUBA': 'CU',
         'CURAÇAO': 'CW',
         'CYPRUS': 'CY',
         'CZECH REPUBLIC': 'CZ',
         'DENMARK': 'DK',
         'DJIBOUTI': 'DJ',
         'DOMINICA': 'DM',
         'DOMINICAN REPUBLIC': 'DO',
         'ECUADOR': 'EC',
         'EGYPT': 'EG',
         'EL SALVADOR': 'SV',
         'EQUATORIAL GUINEA': 'GQ',
         'ERITREA': 'ER',
         'ESTONIA': 'EE',
         'ETHIOPIA': 'ET',
         'FALKLAND ISLANDS (MALVINAS)': 'FK',
         'FAROE ISLANDS': 'FO',
         'FIJI': 'FJ',
         'FINLAND': 'FI',
         'FRANCE': 'FR',
         'FRENCH GUIANA': 'GF',
         'FRENCH POLYNESIA': 'PF',
         'FRENCH SOUTHERN TERRITORIES': 'TF',
         'GABON': 'GA',
         'GAMBIA': 'GM',
         'GEORGIA': 'GE',
         'GERMANY': 'DE',
         'GHANA': 'GH',
         'GIBRALTAR': 'GI',
         'GREECE': 'GR',
         'GREENLAND': 'GL',
         'GRENADA': 'GD',
         'GUADELOUPE': 'GP',
         'GUAM': 'GU',
         'GUATEMALA': 'GT',
         'GUERNSEY': 'GG',
         'GUINEA': 'GN',
         'GUINEA-BISSAU': 'GW',
         'GUYANA': 'GY',
         'HAITI': 'HT',
         'HEARD ISLAND AND MCDONALD ISLANDS': 'HM',
         'HOLY SEE (VATICAN CITY STATE)': 'VA',
         'HONDURAS': 'HN',
         'HONG KONG': 'HK',
         'HUNGARY': 'HU',
         'ICELAND': 'IS',
         'INDIA': 'IN',
         'INDONESIA': 'ID',
         'IRAN, ISLAMIC REPUBLIC OF': 'IR',
         'IRAQ': 'IQ',
         'IRELAND': 'IE',
         'ISLE OF MAN': 'IM',
         'ISRAEL': 'IL',
         'ITALY': 'IT',
         'JAMAICA': 'JM',
         'JAPAN': 'JP',
         'JERSEY': 'JE',
         'JORDAN': 'JO',
         'KAZAKHSTAN': 'KZ',
         'KENYA': 'KE',
         'KIRIBATI': 'KI',
         "KOREA, DEMOCRATIC PEOPLE'S REPUBLIC OF": 'KP',
         'KOREA, REPUBLIC OF': 'KR',
         'KUWAIT': 'KW',
         'KYRGYZSTAN': 'KG',
         "LAO PEOPLE'S DEMOCRATIC REPUBLIC": 'LA',
         'LATVIA': 'LV',
         'LEBANON': 'LB',
         'LESOTHO': 'LS',
         'LIBERIA': 'LR',
         'LIBYA': 'LY',
         'LIECHTENSTEIN': 'LI',
         'LITHUANIA': 'LT',
         'LUXEMBOURG': 'LU',
         'MACAO': 'MO',
         'MACEDONIA, THE FORMER YUGOSLAV REPUBLIC OF': 'MK',
         'MADAGASCAR': 'MG',
         'MALAWI': 'MW',
         'MALAYSIA': 'MY',
         'MALDIVES': 'MV',
         'MALI': 'ML',
         'MALTA': 'MT',
         'MARSHALL ISLANDS': 'MH',
         'MARTINIQUE': 'MQ',
         'MAURITANIA': 'MR',
         'MAURITIUS': 'MU',
         'MAYOTTE': 'YT',
         'MEXICO': 'MX',
         'MICRONESIA, FEDERATED STATES OF': 'FM',
         'MOLDOVA, REPUBLIC OF': 'MD',
         'MONACO': 'MC',
         'MONGOLIA': 'MN',
         'MONTENEGRO': 'ME',
         'MONTSERRAT': 'MS',
         'MOROCCO': 'MA',
         'MOZAMBIQUE': 'MZ',
         'MYANMAR': 'MM',
         'NAMIBIA': 'NA',
         'NAURU': 'NR',
         'NEPAL': 'NP',
         'NETHERLANDS': 'NL',
         'NEW CALEDONIA': 'NC',
         'NEW ZEALAND': 'NZ',
         'NICARAGUA': 'NI',
         'NIGER': 'NE',
         'NIGERIA': 'NG',
         'NIUE': 'NU',
         'NORFOLK ISLAND': 'NF',
         'NORTHERN MARIANA ISLANDS': 'MP',
         'NORWAY': 'NO',
         'OMAN': 'OM',
         'PAKISTAN': 'PK',
         'PALAU': 'PW',
         'PALESTINE, STATE OF': 'PS',
         'PANAMA': 'PA',
         'PAPUA NEW GUINEA': 'PG',
         'PARAGUAY': 'PY',
         'PERU': 'PE',
         'PHILIPPINES': 'PH',
         'PITCAIRN': 'PN',
         'POLAND': 'PL',
         'PORTUGAL': 'PT',
         'PUERTO RICO': 'PR',
         'QATAR': 'QA',
         'RÉUNION': 'RE',
         'ROMANIA': 'RO',
         'RUSSIAN FEDERATION': 'RU',
         'RWANDA': 'RW',
         'SAINT BARTHÉLEMY': 'BL',
         'SAINT HELENA, ASCENSION AND TRISTAN DA CUNHA': 'SH',
         'SAINT KITTS AND NEVIS': 'KN',
         'SAINT LUCIA': 'LC',
         'SAINT MARTIN (FRENCH PART)': 'MF',
         'SAINT PIERRE AND MIQUELON': 'PM',
         'SAINT VINCENT AND THE GRENADINES': 'VC',
         'SAMOA': 'WS',
         'SAN MARINO': 'SM',
         'SAO TOME AND PRINCIPE': 'ST',
         'SAUDI ARABIA': 'SA',
         'SENEGAL': 'SN',
         'SERBIA': 'RS',
         'SEYCHELLES': 'SC',
         'SIERRA LEONE': 'SL',
         'SINGAPORE': 'SG',
         'SINT MAARTEN (DUTCH PART)': 'SX',
         'SLOVAKIA': 'SK',
         'SLOVENIA': 'SI',
         'SOLOMON ISLANDS': 'SB',
         'SOMALIA': 'SO',
         'SOUTH AFRICA': 'ZA',
         'SOUTH GEORGIA AND THE SOUTH SANDWICH ISLANDS': 'GS',
         'SOUTH SUDAN': 'SS',
         'SPAIN': 'ES',
         'SRI LANKA': 'LK',
         'SUDAN': 'SD',
         'SURINAME': 'SR',
         'SVALBARD AND JAN MAYEN': 'SJ',
         'SWAZILAND': 'SZ',
         'SWEDEN': 'SE',
         'SWITZERLAND': 'CH',
         'SYRIAN ARAB REPUBLIC': 'SY',
         'TAIWAN': 'TW',
         'TAJIKISTAN': 'TJ',
         'TANZANIA, UNITED REPUBLIC OF': 'TZ',
         'THAILAND': 'TH',
         'TIMOR-LESTE': 'TL',
         'TOGO': 'TG',
         'TOKELAU': 'TK',
         'TONGA': 'TO',
         'TRINIDAD AND TOBAGO': 'TT',
         'TUNISIA': 'TN',
         'TURKEY': 'TR',
         'TURKMENISTAN': 'TM',
         'TURKS AND CAICOS ISLANDS': 'TC',
         'TUVALU': 'TV',
         'UGANDA': 'UG',
         'UKRAINE': 'UA',
         'UNITED ARAB EMIRATES': 'AE',
         'UNITED KINGDOM': 'GB',
         'UNITED STATES': 'US',
         'UNITED STATES MINOR OUTLYING ISLANDS': 'UM',
         'URUGUAY': 'UY',
         'UZBEKISTAN': 'UZ',
         'VANUATU': 'VU',
         'VENEZUELA, BOLIVARIAN REPUBLIC OF': 'VE',
         'VIETNAM': 'VN',
         'VIET NAM': 'VN',
         'VIRGIN ISLANDS, BRITISH': 'VG',
         'VIRGIN ISLANDS, U.S.': 'VI',
         'WALLIS AND FUTUNA': 'WF',
         'WESTERN SAHARA': 'EH',
         'YEMEN': 'YE',
         'ZAMBIA': 'ZM',
         'ZIMBABWE': 'ZW',
          'AF' : 'AF',
            'AX' : 'AX',
            'AL' : 'AL',
            'DZ' : 'DZ',
            'AS' : 'AS',
            'AD' : 'AD',
            'AO' : 'AO',
            'AI' : 'AI',
            'AQ' : 'AQ',
            'AG' : 'AG',
            'AR' : 'AR',
            'AM' : 'AM',
            'AW' : 'AW',
            'AU' : 'AU',
            'AT' : 'AT',
            'AZ' : 'AZ',
            'BS' : 'BS',
            'BH' : 'BH',
            'BD' : 'BD',
            'BB' : 'BB',
            'BY' : 'BY',
            'BE' : 'BE',
            'BZ' : 'BZ',
            'BJ' : 'BJ',
            'BM' : 'BM',
            'BT' : 'BT',
            'BO' : 'BO',
            'BQ' : 'BQ',
            'BA' : 'BA',
            'BW' : 'BW',
            'BV' : 'BV',
            'BR' : 'BR',
            'IO' : 'IO',
            'BN' : 'BN',
            'BG' : 'BG',
            'BF' : 'BF',
            'BI' : 'BI',
            'KH' : 'KH',
            'CM' : 'CM',
            'CA' : 'CA',
            'CV' : 'CV',
            'KY' : 'KY',
            'CF' : 'CF',
            'TD' : 'TD',
            'CL' : 'CL',
            'CN' : 'CN',
            'CX' : 'CX',
            'CC' : 'CC',
            'CO' : 'CO',
            'KM' : 'KM',
            'CG' : 'CG',
            'CD' : 'CD',
            'CK' : 'CK',
            'CR' : 'CR',
            'CI' : 'CI',
            'HR' : 'HR',
            'CU' : 'CU',
            'CW' : 'CW',
            'CY' : 'CY',
            'CZ' : 'CZ',
            'DK' : 'DK',
            'DJ' : 'DJ',
            'DM' : 'DM',
            'DO' : 'DO',
            'EC' : 'EC',
            'EG' : 'EG',
            'SV' : 'SV',
            'GQ' : 'GQ',
            'ER' : 'ER',
            'EE' : 'EE',
            'ET' : 'ET',
            'FK' : 'FK',
            'FO' : 'FO',
            'FJ' : 'FJ',
            'FI' : 'FI',
            'FR' : 'FR',
            'GF' : 'GF',
            'PF' : 'PF',
            'TF' : 'TF',
            'GA' : 'GA',
            'GM' : 'GM',
            'GE' : 'GE',
            'DE' : 'DE',
            'GH' : 'GH',
            'GI' : 'GI',
            'GR' : 'GR',
            'GL' : 'GL',
            'GD' : 'GD',
            'GP' : 'GP',
            'GU' : 'GU',
            'GT' : 'GT',
            'GG' : 'GG',
            'GN' : 'GN',
            'GW' : 'GW',
            'GY' : 'GY',
            'HT' : 'HT',
            'HM' : 'HM',
            'VA' : 'VA',
            'HN' : 'HN',
            'HK' : 'HK',
            'HU' : 'HU',
            'IS' : 'IS',
            'IN' : 'IN',
            'ID' : 'ID',
            'IR' : 'IR',
            'IQ' : 'IQ',
            'IE' : 'IE',
            'IM' : 'IM',
            'IL' : 'IL',
            'IT' : 'IT',
            'JM' : 'JM',
            'JP' : 'JP',
            'JE' : 'JE',
            'JO' : 'JO',
            'KZ' : 'KZ',
            'KE' : 'KE',
            'KI' : 'KI',
            'KP' : 'KP',
            'KR' : 'KR',
            'KW' : 'KW',
            'KG' : 'KG',
            'LA' : 'LA',
            'LV' : 'LV',
            'LB' : 'LB',
            'LS' : 'LS',
            'LR' : 'LR',
            'LY' : 'LY',
            'LI' : 'LI',
            'LT' : 'LT',
            'LU' : 'LU',
            'MO' : 'MO',
            'MK' : 'MK',
            'MG' : 'MG',
            'MW' : 'MW',
            'MY' : 'MY',
            'MV' : 'MV',
            'ML' : 'ML',
            'MT' : 'MT',
            'MH' : 'MH',
            'MQ' : 'MQ',
            'MR' : 'MR',
            'MU' : 'MU',
            'YT' : 'YT',
            'MX' : 'MX',
            'FM' : 'FM',
            'MD' : 'MD',
            'MC' : 'MC',
            'MN' : 'MN',
            'ME' : 'ME',
            'MS' : 'MS',
            'MA' : 'MA',
            'MZ' : 'MZ',
            'MM' : 'MM',
            'NA' : 'NA',
            'NR' : 'NR',
            'NP' : 'NP',
            'NL' : 'NL',
            'NC' : 'NC',
            'NZ' : 'NZ',
            'NI' : 'NI',
            'NE' : 'NE',
            'NG' : 'NG',
            'NU' : 'NU',
            'NF' : 'NF',
            'MP' : 'MP',
            'NO' : 'NO',
            'OM' : 'OM',
            'PK' : 'PK',
            'PW' : 'PW',
            'PS' : 'PS',
            'PA' : 'PA',
            'PG' : 'PG',
            'PY' : 'PY',
            'PE' : 'PE',
            'PH' : 'PH',
            'PN' : 'PN',
            'PL' : 'PL',
            'PT' : 'PT',
            'PR' : 'PR',
            'QA' : 'QA',
            'RE' : 'RE',
            'RO' : 'RO',
            'RU' : 'RU',
            'RW' : 'RW',
            'BL' : 'BL',
            'SH' : 'SH',
            'KN' : 'KN',
            'LC' : 'LC',
            'MF' : 'MF',
            'PM' : 'PM',
            'VC' : 'VC',
            'WS' : 'WS',
            'SM' : 'SM',
            'ST' : 'ST',
            'SA' : 'SA',
            'SN' : 'SN',
            'RS' : 'RS',
            'SC' : 'SC',
            'SL' : 'SL',
            'SG' : 'SG',
            'SX' : 'SX',
            'SK' : 'SK',
            'SI' : 'SI',
            'SB' : 'SB',
            'SO' : 'SO',
            'ZA' : 'ZA',
            'GS' : 'GS',
            'SS' : 'SS',
            'ES' : 'ES',
            'LK' : 'LK',
            'SD' : 'SD',
            'SR' : 'SR',
            'SJ' : 'SJ',
            'SZ' : 'SZ',
            'SE' : 'SE',
            'CH' : 'CH',
            'SY' : 'SY',
            'TW' : 'TW',
            'TJ' : 'TJ',
            'TZ' : 'TZ',
            'TH' : 'TH',
            'TL' : 'TL',
            'TG' : 'TG',
            'TK' : 'TK',
            'TO' : 'TO',
            'TT' : 'TT',
            'TN' : 'TN',
            'TR' : 'TR',
            'TM' : 'TM',
            'TC' : 'TC',
            'TV' : 'TV',
            'UG' : 'UG',
            'UA' : 'UA',
            'AE' : 'AE',
            'GB' : 'GB',
            'US' : 'US',
            'UM' : 'UM',
            'UY' : 'UY',
            'UZ' : 'UZ',
            'VU' : 'VU',
            'VE' : 'VE',
            'VN' : 'VN',
            'VG' : 'VG',
            'VI' : 'VI',
            'WF' : 'WF',
            'EH' : 'EH',
            'YE' : 'YE',
            'ZM' : 'ZM',
            'ZW' : 'ZW'  }
    
    
    global gcr_data
    global internal_templat
    global summary_df
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None) 
    df = internal_templat
    gateway = pd.read_excel(g_file_path)
    df1 = gateway

    df['Origin  _Origin City'] = df['Origin  _Origin City'].str.upper()
    df['Origin  _Origin State'] = df['Origin  _Origin State'].str.upper()
    df['Destination  _Destination City'] = df['Destination  _Destination City'].str.upper()
    df['Destination  _Destination State'] = df['Destination  _Destination State'].str.upper()
    df['Origin  _Origin State'] = df['Origin  _Origin State'].map(states)
    df['Destination  _Destination State'] = df['Destination  _Destination State'].map(states)

    df['Destination  _Destination Country'].replace('USA', 'US', inplace=True)
    df['Origin  _Origin Country'].replace('USA', 'US', inplace=True)
    pattern=r'([A-Z]){3}'
    #origin gateway correction(removing keyword gateway)       
    df['Origin  _Origin Gateway']=df['Origin  _Origin Gateway'].astype(str).apply(lambda x: re.search(pattern,x)[0] if x and re.search(pattern,x) else np.nan) 
    
        
    #destination gateway keyword removing
    df['Destination  _Destination Gateway'] = df['Destination  _Destination Gateway'].astype(str).apply(lambda x: re.search(pattern, x)[0] if x and re.search(pattern, x) else np.nan)

# ---------------------------------------------------------------------------------------------------------------    
    
    # Origin gateway with state
#     df1_states = df1[['State', 'Gateway']]
#     df1_states['State'] = df1_states['State'].str.upper()
#     df1_states.drop_duplicates('State', keep='last', inplace=True)
#     df = pd.merge(df, df1_states, left_on='Origin  _Origin State', right_on='State', how='left')
#     df['Gateway'] = df['Gateway'].replace('SUB', np.nan)
#     df['Origin  _Origin Gateway'].fillna(df['Gateway'], inplace=True)
#     #df['Origin  _Origin Gateway'].fillna('', inplace=True)
#     del df['State']
#     del df['Gateway']

#     # Origin gateway with city
#     df1_city = df1[[' City/Airport ', 'Gateway']]
#     df1_city[' City/Airport '] = df1_city[' City/Airport '].str.upper()
#     df1_city.drop_duplicates(' City/Airport ', keep='last', inplace=True)
#     df = pd.merge(df, df1_city, left_on='Origin  _Origin City', right_on=' City/Airport ', how='left')
#     df['Origin  _Origin Gateway'].fillna(df['Gateway'], inplace=True)
#     df['Origin  _Origin Gateway'].fillna('', inplace=True)
#     del df[' City/Airport ']
#     del df['Gateway']

# ----------------------------------------------------------------------------------------------------------------
    
    #Origin gateway with Country-city combo
    df['Origin  _Origin Country'] = df['Origin  _Origin Country'].str.lstrip()
    df['Origin  _Origin Country'] = df['Origin  _Origin Country'].str.rstrip()
    df['Origin  _Origin City'] = df['Origin  _Origin City'].str.lstrip()
    df['Origin  _Origin City'] = df['Origin  _Origin City'].str.rstrip()
    df['Origin  _Origin Country'] = df['Origin  _Origin Country'].str.upper()
    df['Origin  _Origin Country_1'] = df['Origin  _Origin Country'].map(country_codes)
    df['Origin  _Origin Key'] = df['Origin  _Origin City']+'-'+df['Origin  _Origin Country_1']
    df1_city_country = df1[['City-Country', 'Gateway']]
    df1_city_country['City-Country'] = df1_city_country['City-Country'].str.upper()
    df1_city_country.drop_duplicates('City-Country', keep='last', inplace=True)
    df = pd.merge(df, df1_city_country, left_on='Origin  _Origin Key', right_on='City-Country', how='left')
    df['Origin  _Origin Gateway'].fillna(df['Gateway'], inplace=True)
    df['Origin  _Origin Gateway'].fillna('', inplace=True)
    del df['Origin  _Origin Country_1']
    del df['Origin  _Origin Key']
    del df['City-Country']
    del df['Gateway']    
    
# -------------------------------------------------------------------------------------------------------------    
     #Destination gateway with Country-city combo
    df['Destination  _Destination Country'] = df['Destination  _Destination Country'].str.lstrip()
    df['Destination  _Destination Country'] = df['Destination  _Destination Country'].str.rstrip()
    df['Destination  _Destination City'] = df['Destination  _Destination City'].str.lstrip()
    df['Destination  _Destination City'] = df['Destination  _Destination City'].str.rstrip()
    df['Destination  _Destination Country'] = df['Destination  _Destination Country'].str.upper()
    df['Destination  _Destination Country_1'] = df['Destination  _Destination Country'].map(country_codes)
    df['Destination  _Destination Key'] = df['Destination  _Destination City']+'-'+df['Destination  _Destination Country_1']
    df1_city_country = df1[['City-Country', 'Gateway']]
    df1_city_country['City-Country'] = df1_city_country['City-Country'].str.upper()
    df1_city_country.drop_duplicates('City-Country', keep='last', inplace=True)
    df = pd.merge(df, df1_city_country, left_on='Destination  _Destination Key', right_on='City-Country', how='left')
    df['Destination  _Destination Gateway'].fillna(df['Gateway'], inplace=True)
    df['Destination  _Destination Gateway'].fillna('', inplace=True)
    del df['Destination  _Destination Country_1']
    del df['Destination  _Destination Key']
    del df['City-Country']
    del df['Gateway'] 
    
#     # Destination gateway using states
#     df1_states = df1[['State', 'Gateway']]
#     df1_states['State'] = df1_states['State'].str.upper()
#     df1_states.drop_duplicates('State', keep='last', inplace=True)
#     df = pd.merge(df, df1_states, left_on='Destination  _Destination State', right_on='State', how='left')
#     df['Gateway'] = df['Gateway'].replace('SUB', np.nan)
#     df['Destination  _Destination Gateway'].fillna(df['Gateway'], inplace=True)
#     del df['State']
#     del df['Gateway']

#     # Destination Gateway using city
#     df1_city = df1[[' City/Airport ', 'Gateway']]
#     df1_city[' City/Airport '] = df1_city[' City/Airport '].str.upper()
#     df1_city.drop_duplicates(' City/Airport ', keep='last', inplace=True)
#     df = pd.merge(df, df1_city, left_on='Destination  _Destination City', right_on=' City/Airport ', how='left')
#     df['Destination  _Destination Gateway'].fillna(df['Gateway'], inplace=True)
#     df['Destination  _Destination Gateway'].fillna('', inplace=True)
#     del df[' City/Airport ']
#     del df['Gateway']

    df1_region = df1[['Region', 'Country']]
    df1_region.drop_duplicates('Country', keep='last', inplace=True)

    # Origin Region
    df['Origin  _Origin Country_1'] = df['Origin  _Origin Country'].map(country_codes)
    df = pd.merge(df, df1_region, left_on='Origin  _Origin Country_1', right_on='Country', how='left')
    df.rename(columns={'Region': 'Origin Region'}, inplace=True)
    df['Origin  _Origin Region'].fillna(df['Origin Region'], inplace=True)
    del df['Country']
    del df['Origin  _Origin Country_1']
#     del df['Origin Region']

#     df = pd.merge(df, df1_region, left_on='Destination  _Destination Country', right_on='Country', how='left')
#     df.rename(columns={'Region': 'Destination Region'}, inplace=True)
#     del df['Country']
    
    # Destination Region
    df['Destination  _Destination Country_1'] = df['Destination  _Destination Country'].map(country_codes)
    df = pd.merge(df, df1_region, left_on='Destination  _Destination Country_1', right_on='Country', how='left')
    df.rename(columns={'Region': 'Destination Region'}, inplace=True)
    df['Destination  _Destination Region'].fillna(df['Destination Region'], inplace=True)
    del df['Country']
    del df['Destination  _Destination Country_1']
#     del df['Destination Region']

#     df = pd.merge(df, df1_region, left_on='Destination  _Destination Country', right_on='Country', how='left')
#     df.rename(columns={'Region': 'Destination Region'}, inplace=True)
#     del df['Country']
         
#     # Origin Region based on gateways
#     df1_OR = df1[['Region', 'Gateway']]
#     df1_OR['Gateway'] = df1_OR['Gateway'].str.upper()
#     df1_OR.drop_duplicates('Gateway', keep='last', inplace=True)
#     df = pd.merge(df, df1_OR, left_on='Origin  _Origin Gateway', right_on='Gateway', how='left')
#     df['Origin  _Origin Region'].fillna(df['Region'], inplace=True)
#     df['Origin  _Origin Region'].fillna('', inplace=True)
#     del df['Region']
#     del df['Gateway']
    
#      # Origin Region based on city/airport
#     df1_OR1 = df1[['Region', ' City/Airport ']]
#     df1_OR1[' City/Airport '] = df1_OR1[' City/Airport '].str.upper()
#     df1_OR1.drop_duplicates(' City/Airport ', keep='last', inplace=True)
#     df = pd.merge(df, df1_OR1, left_on='Origin  _Origin City', right_on=' City/Airport ', how='left')
#     df['Origin  _Origin Region'].fillna(df['Region'], inplace=True)
#     df['Origin  _Origin Region'].fillna('', inplace=True)
#     del df['Region']
#     del df[' City/Airport ']
    
#      # Destination Region based on gateways
#     df1_OR2 = df1[['Region', 'Gateway']]
#     df1_OR2['Gateway'] = df1_OR2['Gateway'].str.upper()
#     df1_OR2.drop_duplicates('Gateway', keep='last', inplace=True)
#     df = pd.merge(df, df1_OR2, left_on='Destination  _Destination Gateway', right_on='Gateway', how='left')
#     df['Destination  _Destination Region'].fillna(df['Region'], inplace=True)
#     df['Destination  _Destination Region'].fillna('', inplace=True)
#     del df['Region']
#     del df['Gateway']
    
#     #Destination Region based on city/airport
#     df1_OR3 = df1[['Region', ' City/Airport ']]
#     df1_OR3[' City/Airport '] = df1_OR3[' City/Airport '].str.upper()
#     df1_OR3.drop_duplicates(' City/Airport ', keep='last', inplace=True)
#     df = pd.merge(df, df1_OR3, left_on='Destination  _Destination City', right_on=' City/Airport ', how='left')
#     df['Destination  _Destination Region'].fillna(df['Region'], inplace=True)
#     df['Destination  _Destination Region'].fillna('', inplace=True)
#     del df['Region']
#     del df[' City/Airport ']
    

    # Country House column
    df['Origin  _Origin Gateway'] = df['Origin  _Origin Gateway'].astype(str)
    df['Destination  _Destination Gateway'] = df['Destination  _Destination Gateway'].astype(str)
    df['origin port'] = df['Origin  _Origin Gateway'].copy()
    df['destination port'] = df['Destination  _Destination Gateway'].copy()
#     df['origin port'] = df['origin port'].str.lstrip()
#     df['origin port'] = df['origin port'].str.rstrip()
#     df['destination port'] = df['destination port'].str.lstrip()
#     df['destination port'] = df['destination port'].str.rstrip()
    df['Lane Information_Country House'] = df.apply(lambda x: country_house_Column(x), axis=1)
    del df['origin port']
    del df['destination port']
    

    column_length = df['Origin  _Origin Country'].count()
    df = df.iloc[:column_length+1, :]
    for index in range(df.shape[0]):
        value = df.loc[index, 'Lane Information_Country House']
        if value.strip() == '':
            df.at[index, 'Country House?'] = 'No'
        else:
            df.at[index, 'Country House?'] = 'Yes'
    print("Gateway Data Transfer Successful!")
    summary_df = df
    df = df.loc[:, :'ALL IN TARGET RATE_COMMENT']
    internal_templat = df
    messagebox.showinfo("Success","Gateway Data Transfer Successful")




In [12]:
def summary_tables():
    save_path = filedialog.asksaveasfilename(defaultextension='.xlsx', initialfile='Summary Tables')
    df = summary_df
    df1 = df.copy()
    df1=df1.rename(columns={'Lane Information_Country House':'Country House','Lane Information_Lane ID':'No of lanes','Shipment Information_Annual Chargeable KGS':'Sum of Chargeable_Weight Kgs','Shipment Information_# of Shipments':'No of shipments','Origin  _Origin Country':'Origin Country','Destination  _Destination Country':'Destination Country','Origin  _Origin Gateway':'Origin Gateway','Destination  _Destination Gateway':'Destination Gateway','Origin  _Origin City':'Origin City','Destination  _Destination City':'Destination City'})
    #df1.loc[~df1["Country House"].isnull(), 'Country House?'] = 'Yes'
    #df1['Country House?'].fillna('No', inplace = True)
    df1['Hazardous (Y/N)']=df1['Shipment Information_Hazardous (Y/N)'].copy()
    df1['General Cargo'] = df1['Hazardous (Y/N)'].copy()
    df1.loc[df1['General Cargo'] == 'N', 'General Cargo'] = 'No'
    df1['General Cargo'] = df1['General Cargo'].str.upper()
    
    #Region to region Table 
    region_to_region = df1.groupby(['Country House?', 'General Cargo', 'Origin Region',
       'Destination Region']).agg({'No of lanes' : 'count', 'Sum of Chargeable_Weight Kgs' : 'sum', 'No of shipments' : 'sum'}, margins = True)
    try:
        region_to_region['% of Chargeable_Weight'] = (region_to_region['Sum of Chargeable_Weight Kgs'] / region_to_region['Sum of Chargeable_Weight Kgs'].sum()) * 100
    except ZeroDivisionError:
        region_to_region['% of Chargeable_Weight'] = 0
    
    region_to_region['% of Chargeable_Weight'] = region_to_region['% of Chargeable_Weight'].apply(lambda x: round(x, 2))
    try:
        region_to_region['Average weight per shipment (Kgs)'] = (region_to_region['Sum of Chargeable_Weight Kgs'] / (region_to_region['No of shipments'])).astype(int).round()
    except ZeroDivisionError:
        region_to_region['Average weight per shipment (Kgs)'] = 0
        
        
    #Country to country Table
    country_to_country_df = df1.groupby(['Country House?', 'General Cargo', 'Origin Country',
       'Destination Country']).agg({'No of lanes' : 'count', 'Sum of Chargeable_Weight Kgs' : 'sum', 'No of shipments' : 'sum'})
    
    try:
        country_to_country_df['% of Chargeable_Weight'] = (country_to_country_df['Sum of Chargeable_Weight Kgs']/ country_to_country_df['Sum of Chargeable_Weight Kgs'].sum())*100
    except ZeroDivisionError:
        country_to_country_df['% of Chargeable_Weight'] = 0
    country_to_country_df['% of Chargeable_Weight'] = country_to_country_df['% of Chargeable_Weight'].apply(lambda x: round(x, 2))
    try:
        country_to_country_df['Average weight per shipment (Kgs)'] = (country_to_country_df['Sum of Chargeable_Weight Kgs'] / (country_to_country_df['No of shipments'])).astype(int).round()
    except ZeroDivisionError:
        country_to_country_df['Average weight per shipment (Kgs)'] = 0 
        
        
    #Gateway to gateway
    gateway_to_gateway_df = df1.groupby(['Country House?', 'General Cargo', 'Origin Gateway',
       'Destination Gateway']).agg({'No of lanes' : 'count', 'Sum of Chargeable_Weight Kgs' : 'sum', 'No of shipments' : 'sum'})
    try:
        gateway_to_gateway_df['% of Chargeable_Weight'] = (gateway_to_gateway_df['Sum of Chargeable_Weight Kgs']/ gateway_to_gateway_df['Sum of Chargeable_Weight Kgs'].sum())*100
    except ZeroDivisionError:
        gateway_to_gateway_df['% of Chargeable_Weight'] = 0
    gateway_to_gateway_df['% of Chargeable_Weight'] = gateway_to_gateway_df['% of Chargeable_Weight'].apply(lambda x : round(x,2))
    try:
        gateway_to_gateway_df['Average weight per shipment (Kgs)'] = (gateway_to_gateway_df['Sum of Chargeable_Weight Kgs']/(gateway_to_gateway_df['No of shipments'])).astype(int).round()
    except ZeroDivisionError:
        gateway_to_gateway_df['Average weight per shipment (Kgs)'] = 0

    #Origin city to Destination City
    og_city_to_dest_city_df = df1.groupby(['Country House?', 'General Cargo',  'Origin Country',
       'Destination Country','Origin City',
       'Destination City']).agg({'No of lanes' : 'count', 'Sum of Chargeable_Weight Kgs' : 'sum', 'No of shipments' : 'sum'})
    try:
        og_city_to_dest_city_df['% of Chargeable_Weight'] = (og_city_to_dest_city_df['Sum of Chargeable_Weight Kgs']/ og_city_to_dest_city_df['Sum of Chargeable_Weight Kgs'].sum())*100
    except ZeroDivisionError:
        og_city_to_dest_city_df['% of Chargeable_Weight'] = 0
    og_city_to_dest_city_df['% of Chargeable_Weight'] = og_city_to_dest_city_df['% of Chargeable_Weight'].apply(lambda x: round(x, 2))
    try:
        og_city_to_dest_city_df['Average weight per shipment (Kgs)'] = (og_city_to_dest_city_df['Sum of Chargeable_Weight Kgs']/(og_city_to_dest_city_df['No of shipments'])).astype(int).round()
    except ZeroDivisionError:
        og_city_to_dest_city_df['Average weight per shipment (Kgs)'] = 0 

    #Origin
    origin_df = df1.groupby([  'Origin Country',
      'Origin Gateway','Origin City',
       ]).agg({'No of lanes' : 'count', 'Sum of Chargeable_Weight Kgs' : 'sum', 'No of shipments' : 'sum'})
    
    try:
        origin_df['% of Chargeable_Weight'] = (origin_df['Sum of Chargeable_Weight Kgs']/ origin_df['Sum of Chargeable_Weight Kgs'].sum())*100
    except ZeroDivisionError:
        origin_df['% of Chargeable_Weight'] = 0 
    origin_df['% of Chargeable_Weight'] = origin_df['% of Chargeable_Weight'].apply(lambda x: round(x, 2))
    try:
        origin_df['Average weight per shipment (Kgs)'] = (origin_df['Sum of Chargeable_Weight Kgs']/(origin_df['No of shipments'] )).astype(int).round()
    except ZeroDivisionError:
        origin_df['Average weight per shipment (Kgs)'] = 0 
        
    #Destination
    destination_df = df1.groupby([  'Destination Country',
      'Destination Gateway','Destination City',
       ]).agg({'No of lanes' : 'count', 'Sum of Chargeable_Weight Kgs' : 'sum', 'No of shipments' : 'sum'})
    try:
        destination_df['% of Chargeable_Weight'] = (destination_df['Sum of Chargeable_Weight Kgs']/ destination_df['Sum of Chargeable_Weight Kgs'].sum())*100
    except ZeroDivisionError:
        destination_df['% of Chargeable_Weight'] = 0
    destination_df['% of Chargeable_Weight'] = destination_df['% of Chargeable_Weight'].apply(lambda x : round(x,2))
    try:
        destination_df['Average weight per shipment (Kgs)'] = (destination_df['Sum of Chargeable_Weight Kgs']/(destination_df['No of shipments'])).astype(int).round()
    except ZeroDivisionError:
        destination_df['Average weight per shipment (Kgs)'] = 0

    with pd.ExcelWriter(save_path) as writer:
        
    # Write each dataframe to a separate sheet
        region_to_region.to_excel(writer, sheet_name='Region to Regrion')
        country_to_country_df.to_excel(writer, sheet_name='Country to Country')
        gateway_to_gateway_df.to_excel(writer, sheet_name='Gateway to Gateway')
        og_city_to_dest_city_df.to_excel(writer, sheet_name='Origin city to Destion city')
        origin_df.to_excel(writer, sheet_name='Origin')
        destination_df.to_excel(writer, sheet_name='Destination')  
    print("Summary Table saved successfully.")
    messagebox.showinfo("Success", "Summary Table saved successfully")


In [13]:
def download_excel():
    file_path = filedialog.asksaveasfilename(defaultextension='.xlsx')
    new_columns = [tuple(col.split('_')) for col in  internal_templat.columns]
    internal_templat.columns = pd.MultiIndex.from_tuples(new_columns)
    internal_templat.dropna(how='all',inplace=True)   
    internal_templat.reset_index(drop=True, inplace=True)
    l=len(internal_templat)
    index=pd.Index(range(l),name='index')
    internal_templat.index=index
    internal_templat.to_excel(file_path)
    start_col = len(internal_templat.columns) + 2 
    # Load the workbook
    workbook = openpyxl.load_workbook(file_path)
    sheet = workbook.active
    
    for c_idx, (col_name, values) in enumerate(remaining_columns.items()):
        sheet.cell(row=1, column=start_col + c_idx, value=col_name)  # Write column name
    
         # Iterate over values and write them in the same column
        for r_idx, value in enumerate(values):
            sheet.cell(row=r_idx + 4, column=start_col + c_idx, value=value)



    # Set the row height for the first row to 73.5
    sheet.row_dimensions[1].height = 73.5

    # Set the row height for the second row to 48
    sheet.row_dimensions[2].height = 48
    
    # Set wrap text for the first row
    for row in sheet.iter_rows(min_row=1, max_row=1):
        for cell in row:
            cell.alignment = Alignment(wrap_text=True, horizontal='center', vertical='center')
    
    # Apply formatting to row 1
    font_row1 = Font(name='Calibri', size=14, bold=True)
    for row in sheet.iter_rows(min_row=1, max_row=1):
        for cell in row:
            cell.font = font_row1
            
    # Apply formatting to row 2
    font_row2 = Font(name='Calibri', size=12, bold=True)
    for row in sheet.iter_rows(min_row=2, max_row=2):
        for cell in row:
            cell.font = font_row2

    # Colors for cells
    color_cells(sheet, 'B1:G1', 'D9D9D9')
    color_cells(sheet, 'B2:G2', 'D9D9D9')
    color_cells(sheet, 'H1:N1', '95B3D7')
    color_cells(sheet, 'H2:N2', '95B3D7')
    color_cells(sheet, 'O1:U1', 'B1A0C7')
    color_cells(sheet, 'O2:U2', 'B1A0C7')
    color_cells(sheet, 'V1:AN1', 'BFBFBF')
    color_cells(sheet, 'V2:AN2', 'BFBFBF')
    color_cells(sheet, 'AO1:DY1', '95B3D7')
    color_cells(sheet, 'AO2:DY2', '95B3D7')
    color_cells(sheet, 'DZ1:FZ1', 'A6A6A6')
    color_cells(sheet, 'DZ2:EH2', 'A6A6A6')
    color_cells(sheet, 'EI2:EV2', 'FFFF00')
    color_cells(sheet, 'EW2:FZ2', 'A6A6A6')
    color_cells(sheet, 'GA1:JI1', 'B1A0C7')
    color_cells(sheet, 'GA2:JI2', 'B1A0C7')
    color_cells(sheet, 'JJ1:JQ1', 'E6B8B7')
    color_cells(sheet, 'JJ2:JQ2', 'E6B8B7')
    # Adjust column widths based on the content
    for column in sheet.columns:
        max_length = 0
        for cell in column:
            if cell.coordinate in sheet.merged_cells:
                continue
            value = cell.value
            if value:
                cell_length = len(str(value))
                if cell_length > max_length:
                    max_length = cell_length
        
        adjusted_width = (max_length + 2) * 1.2
        column_letter = openpyxl.utils.get_column_letter(column[0].column)
        sheet.column_dimensions[column_letter].width = adjusted_width
    
               
    # Save the updated workbook
    workbook.save(file_path)
    print("Excel Saved Successfully")
    messagebox.showinfo("Success", "Excel Saved Successfully")
    pass




In [14]:
def color_cells(sheet, cell_range, fill_color):
    fill = PatternFill(start_color=fill_color, end_color=fill_color, fill_type="solid")
    
    for row in sheet[cell_range]:
        for cell in row:
            cell.fill = fill

In [15]:
def download_excel_2():
    global global_destination_file
    file_path = filedialog.asksaveasfilename(defaultextension='.xlsx')
    new_columns = [tuple(col.split('_')) for col in  processed_dat.columns]
    processed_dat.columns = pd.MultiIndex.from_tuples(new_columns)
    processed_dat.dropna(how='all',inplace=True)   
    processed_dat.reset_index(drop=True, inplace=True)
    l=len(processed_dat)
    index=pd.Index(range(l),name='index')
    processed_dat.index=index
    processed_dat.to_excel(file_path)
    global_destination_file = file_path
    messagebox.showinfo("Success", "Excel Saved Successfully")
    pass

In [16]:
def get_source_column_names(source_worksheet, source_br_column_index):
    source_column_names = []
    add_columns = False

    for col_index, cell in enumerate(source_worksheet[1], start=1):
        if add_columns:
            source_column_names.append(cell.value)
        if col_index == source_br_column_index+1:
            add_columns = True

            
    return source_column_names

def customer_template_format():
    global global_destination_file
    global c_file_path
    global last_column_index
    

    source_start_row = int(num_rows_entry_1.get())
    destination_start_row = int(skip_rows_entry_1.get())
    sheet_name_1 = sheet_entry_.get()
    skip_rows = int(skip_rows_entry_.get())
    

    # Load the source and destination workbooks
    dest_wb = openpyxl.load_workbook(c_file_path)
    source_wb = openpyxl.load_workbook(global_destination_file)

    # Get the first sheet from each workbook
    source_worksheet = source_wb.active
    if not sheet_name_1:
        destination_worksheet = dest_wb.active  # Use the default active sheet
    else:
        destination_worksheet = dest_wb[sheet_name_1]

    # Get the index of the last column
    #last_column_index = destination_worksheet.max_column - 1
   
    # Delete the first column (column A) in the source worksheet
    source_worksheet.delete_cols(1)

    # Clear the destination data, formatting, and styles from the specified row onwards
    for row_index in range(destination_start_row, destination_worksheet.max_row + 1):
        for col_index in range(1, destination_worksheet.max_column + 1):
            #destination_worksheet.cell(row=row_index, column=col_index, value=np.nan)
            destination_worksheet.cell(row=row_index, column=col_index).font = openpyxl.styles.Font()
            destination_worksheet.cell(row=row_index, column=col_index).border = openpyxl.styles.Border()
            destination_worksheet.cell(row=row_index, column=col_index).fill = openpyxl.styles.PatternFill()
            
    for row_index, source_row in enumerate(source_worksheet.iter_rows(min_row=source_start_row), start=destination_start_row):
        for col_index, cell in enumerate(source_row, start=1):
            destination_worksheet.cell(row=row_index, column=col_index, value=cell.value)
            

    #source_br_column_index = last_column_index("BP")

    if last_column_index is not None:
      
        destination_start_column = last_column_index + 1

       
        source_column_names = get_source_column_names(source_worksheet, last_column_index)
        for col_index, column_name in enumerate(source_column_names, start=destination_start_column+1):
            destination_worksheet.cell(row=skip_rows+ 1, column=col_index, value=column_name)
    else:
        print("Column not found in the source worksheet.")

    # Save the destination workbook
    dest_wb.save(c_file_path)
    print("Transfer Successful")
    messagebox.showinfo("Success", "Customer Template Updated Successful")

In [17]:
def clear_entry():
    global file_path, c_file_path,g_file_path
    skip_rows_entry.delete(0, 'end')
    num_rows_entry.delete(0, 'end')
    sheet_entry.delete(0, 'end')
    file_path = None
    c_file_path = None
    g_file_path = None
    messagebox.showinfo("Reset Success", "Upload all the files again")

In [18]:
def clear_entry_1():
    global file_path, c_file_path,file_path_1
    skip_rows_entry_.delete(0, 'end')
    num_rows_entry_.delete(0, 'end')
    sheet_entry_.delete(0, 'end')
    file_path = None
    c_file_path = None
    file_path_1 = None
    messagebox.showinfo("Reset Success", "Upload all the files again")

In [19]:
# Function to open the main UI when the user logs in
def open_main_ui():
    # Close the login window
    root.destroy()

In [20]:
root = customtkinter.CTk()
width=root.winfo_screenwidth()
height=root.winfo_screenheight()
# window.geometry("%dx%d"%(w,h))


root.title("Login Page")

# x = (root.winfo_screenwidth() // 2) - (width //2 )
# y = (root.winfo_screenwidth() // 3) - (height //2 )

# root.geometry('{} x {} + {}'.format(width,height,x,y))
root.geometry("%dx%d"%(width,height))
root.resizable(True,True)


bg_image = customtkinter.CTkImage(Image.open('background_image_1.png'), size = (width,height))
bg_image_label = customtkinter.CTkLabel(root, image = bg_image, text = "" )
bg_image_label.grid(row = 0, column = 0, padx = (0,120), pady = (0,70))


login_btn = customtkinter.CTkButton(root, text = "START", width = 200, height = 60,command = open_main_ui)
login_btn.grid(row = 0, column = 0, padx= (0,600), pady = (220,100))


root.mainloop()


In [ ]:
import tkinter as tk
from PIL import Image, ImageTk



# Create the main window

customtkinter.set_appearance_mode("light")
customtkinter.set_default_color_theme("dark-blue")
window = customtkinter.CTk()
my_image = customtkinter.CTkImage(light_image=Image.open("Pict.png"),
                                  dark_image=Image.open("Pict.png"),
                                  size=(60,30))
window.wm_iconbitmap('Picture3.ico')
#window.geometry("1536x864")
window.title("Conversion Tool.py")
#window.attributes('-fullscreen',True)
w=window.winfo_screenwidth()
h=window.winfo_screenheight()
window.geometry("%dx%d"%(w,h))
customtkinter.set_widget_scaling(0.8)

window.state("zoomed")
window.grid_rowconfigure((0,1,2), weight=1)
window.grid_columnconfigure((0,1,2), weight=1)

# bg_image1 = customtkinter.CTkImage(Image.open('Pict - Copy.png'), size = (width,height))
# bg_image_label_1 = customtkinter.CTkLabel(window, image = bg_image1, text = "" )
# bg_image_label_1.grid(row = 0, column = 0, padx = (0,120), pady = (0,70))


tabview = customtkinter.CTkTabview(master=window,border_width=2,border_color="#FF6600")
#tabview.grid_rowconfigure((0,1,2), weight=1)
tabview.grid(row=0,sticky='nsew',padx=20,pady=20,rowspan=3,columnspan=3)

# Create a Canvas for Pre Analysis
# canvas_tab1 = tk.Canvas(tabview.get_tab("Pre Analysis"))



tabview.add("Pre Analysis")  # add tab at the end
tabview.add("Post Analysis")  # add tab at the end
tabview.set("Pre Analysis")  # set currently visible tab






frame_1 = customtkinter.CTkFrame(master=tabview.tab("Pre Analysis"))
frame_1.grid(row=0,sticky='nsew',padx=20,pady=10)
label_1 = customtkinter.CTkLabel(frame_1, text="PART A - CONFIGURE",text_color="#4D148C",font=("Helvetica",10,"underline","bold"))
label_1.grid(row=0, sticky="w",padx=10)

frame_2 = customtkinter.CTkScrollableFrame(master=tabview.tab("Pre Analysis"), orientation="vertical",label_text="PART B    ATTRIBUTES",label_font=("Helvetica",10,"underline","bold"),width=950, height=800)
frame_2.grid(row=0, column=1, padx=30, pady=10,sticky='nsew',rowspan=9)



frame_3=customtkinter.CTkFrame(master=tabview.tab("Pre Analysis"))
frame_3.grid(row=1,column=0,sticky='nsew',padx=20,pady=10)
label_3 = customtkinter.CTkLabel(frame_3, text="PART C - COMPILE",text_color="#4D148C",font=("Helvetica",10,"underline","bold"))
label_3.grid(row=0, sticky="w",padx=10)

frame_4=customtkinter.CTkFrame(master=tabview.tab("Pre Analysis"))
frame_4.grid(row=2,sticky='nsew',padx=20,pady=10)
label_4 = customtkinter.CTkLabel(frame_4, text="PART D - DOWNLOADS",text_color="#4D148C",font=("Helvetica",10,"underline","bold"))
label_4.grid(row=0, sticky="w",padx=10)


frame_5=customtkinter.CTkFrame(master=tabview.tab("Pre Analysis"))
frame_5.grid(row=3,sticky='nsew',padx=20,pady=10)
label_5 = customtkinter.CTkLabel(frame_5, text="PART E -  UPLOAD",text_color="#4D148C",font=("Helvetica",10,"underline","bold"))
label_5.grid(row=0, sticky="w",padx=10)


frame_6=customtkinter.CTkFrame(master=tabview.tab("Pre Analysis"))
frame_6.grid(row=4,sticky='nsew',padx=20,pady=10)


frame_1_ = customtkinter.CTkFrame(master=tabview.tab("Post Analysis"))
frame_1_.grid(row=0,sticky='nsew',padx=20,pady=20)
label_1_ = customtkinter.CTkLabel(frame_1_, text="PART A - CONFIGURE",text_color="#4D148C",font=("Helvetica",10,"underline","bold"))
label_1_.grid(row=0, sticky="w",padx=10)

frame_2_ = customtkinter.CTkScrollableFrame(master=tabview.tab("Post Analysis"), label_text="PART B -  ATTRIBUTES",label_font=("Helvetica",10,"underline","bold"),orientation="vertical",width=950, height=800)
frame_2_.grid(row=0, column=1, padx=30, pady=20,sticky='nsew',rowspan=9)

frame_3_=customtkinter.CTkFrame(master=tabview.tab("Post Analysis"))
frame_3_.grid(row=1,sticky='nsew',padx=20,pady=20)
label_3_ = customtkinter.CTkLabel(frame_3_, text="PART C  - COMPILE",text_color="#4D148C",font=("Helvetica",10,"underline","bold"))
label_3_.grid(row=0, sticky="w",padx=10)

frame_4_=customtkinter.CTkFrame(master=tabview.tab("Post Analysis"))
frame_4_.grid(row=2,sticky='nsew',padx=20,pady=20)
label_4_ = customtkinter.CTkLabel(frame_4_, text="PART D  - DOWNLOADS",text_color="#4D148C",font=("Helvetica",10,"underline","bold"))
label_4_.grid(row=0, sticky="w",padx=10)

frame_5_=customtkinter.CTkFrame(master=tabview.tab("Post Analysis"))
frame_5_.grid(row=3,sticky='nsew',padx=20,pady=20)
label_5_ = customtkinter.CTkLabel(frame_5_, text="PART E  - TRANSFER TO CUSTOMER TEMPLATE",text_color="#4D148C",font=("Helvetica",10,"underline","bold"))
label_5_.grid(row=0, sticky="w",padx=10)


heading_i=customtkinter.CTkLabel(tabview,image=my_image,text="",text_color="#999999",font=("Arial",16,"bold"))
heading_i.grid(row=0, column=0, sticky="w", padx=10, pady=3)

# ===============================Button Image=================================================
button_image = customtkinter.CTkImage(Image.open("excelimage1.png"), size=(24, 24))
transfer_image = customtkinter.CTkImage(Image.open("transfer_image.png"), size=(24, 24))
download_image = customtkinter.CTkImage(Image.open("downloadimage.png"), size=(24, 24))


# ===========================================================================================


# Choose Internal Template button
internal_template_button_ = customtkinter.CTkButton(frame_1_,fg_color="#4D148C",border_color="#FF6600",border_width=2, text="Working Internal Template", command=choose_internal_template,
                                     width=28, anchor="center", image = button_image)
internal_template_button_.grid(row=1, column=1, sticky="w", padx=10, pady=3)

internal_template_button_og = customtkinter.CTkButton(frame_1_,fg_color="#4D148C",border_color="#FF6600",border_width=2, text="Internal Template", command=choose_internal_template_1,
                                     width=28, anchor="center", image = button_image)
internal_template_button_og.grid(row=1, column=0, sticky="w", padx=10, pady=3)

# Choose Customer Template button
customer_template_button_ = customtkinter.CTkButton(frame_1_,fg_color="#4D148C", border_color="#FF6600",border_width=2,text="Customer Template", command=choose_customer_template,
                                     width=28, anchor="center", image = button_image)
customer_template_button_.grid(row=1, column=2, sticky="w", padx=10, pady=3)


# Label and Entry for number of rows to skip
label_s_entry_ = customtkinter.CTkLabel(frame_1_, text="Rows to Skip:")
label_s_entry_.grid(row=2, column=0, sticky="w",padx=10, pady=3)

skip_rows_entry_ = customtkinter.CTkEntry(frame_1_,border_color="#FF6600",border_width=2, width=34)
skip_rows_entry_.grid(row=2, column=1, sticky="w", padx=10, pady=3)

# Label and Entry for number of headers
label_n_entry_ = customtkinter.CTkLabel(frame_1_, text="Rows to Combine:")
label_n_entry_.grid(row=2, column=2, sticky="w",padx=10, pady=3)

num_rows_entry_ = customtkinter.CTkEntry(frame_1_,border_color="#FF6600",border_width=2, width=34)
num_rows_entry_.grid(row=2, column=3, sticky="w", padx=10, pady=3)

# Label and Entry for sheet name
label_sheet_entry_ = customtkinter.CTkLabel(frame_1_, text="Sheet Name:")
label_sheet_entry_.grid(row=3, column=0, sticky="w",padx=10, pady=3)

sheet_entry_ = customtkinter.CTkEntry(frame_1_,border_color="#FF6600",border_width=2, width=100)
sheet_entry_.grid(row=3, column=1, sticky="w", padx=10, pady=20)

refresh_button = customtkinter.CTkButton(frame_1_,fg_color="#4D148C", text="Reset", command=clear_entry_1, width=24, anchor="center")
refresh_button.grid(row=5, column=1, sticky="ew", padx=10, pady=3)

# transfer_matches_button = customtkinter.CTkButton(frame_3_, fg_color="#4D148C", text="Load Matches", command=load_selected_matches, width=28, anchor="center")
# transfer_matches_button.grid(row=5, column=1, sticky="w", padx=10, pady=3)


# Show Columns button
# transfer_button_ = customtkinter.CTkButton(frame_4_,fg_color="#4D148C", text="Show Columns", command=transfer_data_2, width=28, anchor="center")
# transfer_button_.grid(row=7, column=0, sticky="w", padx=10, pady=3)

transfer_button_ = customtkinter.CTkButton(frame_3_, fg_color="#4D148C", text="RFQ Attributes", command = threading.Thread(target=transfer_data_2).start, width=28, anchor="center")
transfer_button_.grid(row=1, column=0, sticky="w", padx=10, pady=3)


# transfer_button = customtkinter.CTkButton(frame_3,fg_color="#4D148C", text="RFQ Attributes", command=transfer_data, width=28, anchor="center")
# transfer_button.grid(row=0, column=0, sticky="nsew", padx=10, pady=3)

 # Transfer button for all columns
transfer_all_button = customtkinter.CTkButton(frame_3_,fg_color="#4D148C", text="Convert(IT-CT)", command=transfer_all_data_2, width=28, anchor="center",image =  transfer_image)
transfer_all_button.grid(row=1, column=2, sticky="w", padx=10, pady=3)



transfer_all_button = customtkinter.CTkButton(frame_3_,fg_color="#4D148C", text=" Transfer Additonal Columns", command=additional_columns, width=28, anchor="center",image =  transfer_image)
transfer_all_button.grid(row=1, column=3, sticky="w", padx=10, pady=3)
# # Transfer Gateway Data button
# transfer_all_button = customtkinter.CTkButton(frame_4_, text="Transfer Gateway Data", command=truck_df_treatment, width=28, anchor="center")
# transfer_all_button.grid(row=7, column=2, sticky="w", padx=10, pady=3)

download_button = customtkinter.CTkButton(frame_4_,fg_color="#4D148C", text="Download Customer Template", command=download_excel_2, width=28, anchor="center", image = download_image)
download_button.grid(row=1, column=0, sticky="w", padx=10, pady=3)

# Label and Entry for number of headers
label_n_entry_ = customtkinter.CTkLabel(frame_5_, text="Enter the row number to start copying from:")
label_n_entry_.grid(row=3, column=0, sticky="w",padx=10, pady=3)

num_rows_entry_1 = customtkinter.CTkEntry(frame_5_, width=34)
num_rows_entry_1.grid(row=3, column=1, sticky="w", padx=10, pady=3)

# Label and Entry for number of rows to skip in final customer template
label_n_entry_ = customtkinter.CTkLabel(frame_5_, text="Enter the row number to start pasting to:")
label_n_entry_.grid(row=4, column=0, sticky="w",padx=10, pady=3)

skip_rows_entry_1 = customtkinter.CTkEntry(frame_5_, width=34)
skip_rows_entry_1.grid(row=4, column=1, sticky="w", padx=10, pady=3)

download_button = customtkinter.CTkButton(frame_5_,fg_color="#4D148C",text="Update Customer Template", command=customer_template_format, width=28, anchor="center")
download_button.grid(row=5, column=0, sticky="w", padx=10, pady=3)



# heading_l=tk.Label(window,text="Bid Processing App",font=("Arial",16,"bold"))
# heading_l.grid(row=0, column=0, sticky="w", padx=10, pady=3)

#TAB1

# Choose Internal Template button
internal_template_button = customtkinter.CTkButton(frame_1,fg_color="#4D148C", text="Internal Template",border_color="#FF6600",border_width=2, command=choose_internal_template,
                                     width=28, anchor="center", image = button_image)
internal_template_button.grid(row=1, column=0, sticky="w", padx=10, pady=3)

# Choose Customer Template button
customer_template_button = customtkinter.CTkButton(frame_1,fg_color="#4D148C", text="Customer Template",border_color="#FF6600",border_width=2, command=choose_customer_template,
                                     width=28, anchor="center", image = button_image)
customer_template_button.grid(row=1, column=1, sticky="w", padx=10, pady=3)


#gateway Button

customer_template_button = customtkinter.CTkButton(frame_1,fg_color="#4D148C", text="Gateway", border_color="#FF6600",border_width=2,
                                                   command=choose_gateway_file,
                                     width=28, anchor="center", image = button_image)
customer_template_button.grid(row=1, column=2, sticky="w", padx=10, pady=3)

# Label and Entry for number of rows to skip
label_s_entry = customtkinter.CTkLabel(frame_1, text="Rows to Skip:")
label_s_entry.grid(row=2, column=0, sticky="w",padx=10, pady=1)

skip_rows_entry = customtkinter.CTkEntry(frame_1,border_color="#FF6600",border_width=2, width=34)
skip_rows_entry.grid(row=2, column=1, sticky="w",padx=10, pady=1 )

# Label and Entry for number of headers
label_n_entry = customtkinter.CTkLabel(frame_1, text="Rows to Combine:")
label_n_entry.grid(row=2, column=2, sticky="w",padx=10, pady=3)

num_rows_entry = customtkinter.CTkEntry(frame_1,border_color="#FF6600",border_width=2, width=34)
num_rows_entry.grid(row=2, column=3, sticky="w", padx=10, pady=3)

# Label and Entry for sheet name
label_sheet_entry = customtkinter.CTkLabel(frame_1, text="Sheet Name:")
label_sheet_entry.grid(row=3, column=0, sticky="w",padx=10, pady=3)

sheet_entry = customtkinter.CTkEntry(frame_1,border_color="#FF6600",border_width=2, width=100)
sheet_entry.grid(row=3, column=1, sticky="w", padx=10, pady=3)

refresh_button = customtkinter.CTkButton(frame_1,fg_color="#4D148C", text="Reset", command=clear_entry, width=24, anchor="center")
refresh_button.grid(row=5, column=1, sticky="ew", padx=10, pady=3)

# Show Columns button

transfer_button = customtkinter.CTkButton(frame_3,fg_color="#4D148C", text="RFQ Attributes", command = threading.Thread(target=transfer_data).start, width=28, anchor="center")
transfer_button.grid(row=1, column=0, sticky="nsew", padx=10, pady=3)

 # Transfer button for all columns
transfer_all_button = customtkinter.CTkButton(frame_3,fg_color="#4D148C", text="Convert(CT-IT)", command=threading.Thread(target=transfer_all_data).start , width=28, anchor="center", image = transfer_image)
transfer_all_button.grid(row=1, column=1, sticky="nsew", padx=10, pady=3)

# Transfer Gateway Data button
transfer_all_button = customtkinter.CTkButton(frame_3,fg_color="#4D148C", text="Update Gateway and Regions", command=truck_df_treatment, width=28, anchor="center", image = transfer_image)
transfer_all_button.grid(row=1, column=2, sticky="nsew", padx=10, pady=3)

# Download Summary Excel button
download_button = customtkinter.CTkButton(frame_4,fg_color="#4D148C", text="Summary Table", command=summary_tables, width=28, anchor="center", image = download_image)
download_button.grid(row=2, column=0, sticky="ew", padx=10, pady=3)

# Download Excel button
download_button = customtkinter.CTkButton(frame_4,fg_color="#4D148C", text="Internal Template", command=download_excel, width=28, anchor="center", image = download_image)
download_button.grid(row=2, column=1, sticky="w", padx=10, pady=3)


# # Create a new button in "Frame2" of "Pre Analysis" to transfer selected matches
# transfer_matches_button = customtkinter.CTkButton(frame_3_,fg_color="#4D148C", text="Transfer Matches", command=transfer_selected_matches, width=28, anchor="center")
# transfer_matches_button.grid(row=5, column=1, sticky="w", padx=10, pady=3)
transfer_matches_button = customtkinter.CTkButton(frame_3_, fg_color="#4D148C", text="Load Matches", command=load_selected_matches, width=28, anchor="center")
transfer_matches_button.grid(row=1, column=1, sticky="w", padx=10, pady=3)



# Upload Matches button
upload_matches_button = customtkinter.CTkButton(frame_5,fg_color="#4D148C", text="Pairing", command=upload_matches, width=28, anchor="center")
upload_matches_button.grid(row=1, column=0, sticky="ew", padx=10, pady=3)

d_button = customtkinter.CTkButton(frame_5,fg_color="#4D148C", text="RFQ Mapping", command=transfer_all_data_3, width=28, anchor="center")
d_button.grid(row=1, column=1, sticky="w", padx=10, pady=3)

# Download Matches button
download_matches_button = customtkinter.CTkButton(frame_5,fg_color="#4D148C", text="Store Mapping", command=save_selected_matches, width=28, anchor="center")
download_matches_button.grid(row=1, column=2, sticky="w", padx=10, pady=3)

appearance_mode_optionemenu = customtkinter.CTkOptionMenu(frame_6,fg_color="#4D148C" ,values=["Light" ,"Dark","System"],anchor="center" ,command=change_appearance_mode_event,width=28)
appearance_mode_optionemenu.grid(row=0, column=2,sticky="nsew", padx=10, pady=3)

scaling_optionemenu = customtkinter.CTkOptionMenu(frame_6,fg_color="#4D148C", values=["80%", "85%","90%","100%", "110%", "120%"], anchor="center" ,command=change_scaling_event,width=28)
scaling_optionemenu.grid(row=0, column=3,sticky="nsew", padx=10, pady=3)
scaling_optionemenu.set("80%")







# Run the main window loop
window.mainloop()     

# # Create the login window
# login_window = tk.Tk()
# login_window.title("Login")
# login_window.geometry("400x200")

# # Add a label for the username
# username_label = tk.Label(login_window, text="Username:")
# username_label.pack()

# # Add an entry widget for the username
# username_entry = tk.Entry(login_window)
# username_entry.pack()

# # Add a label for the password
# password_label = tk.Label(login_window, text="Password:")
# password_label.pack()

# # Add an entry widget for the password (you can replace this with a password entry widget for security)
# password_entry = tk.Entry(login_window, show="*")
# password_entry.pack()

# # Add a button to log in and open the main UI
# login_button = tk.Button(login_window, text="Login", command=open_main_ui)
# login_button.pack()

# # Add any other widgets you need for the login screen

# # Start the main loop for the login window
# login_window.mainloop()





Internal template loaded successfully!
Transfer successful
Gateway Data Transfer Successful!
Summary Table saved successfully.
Excel Saved Successfully
